### This code plots the 1D and 2D kinematics and differences in kinematics for lep1 and lep2 in $Z \rightarrow \ell\ell$ events. It has a few different switches:

### FIXME:
1. `make_plain_kinem_plots` = A basic kinematic plotter to show the $\eta, p_{T}, \phi, \Delta \mathrm{R}$ for lep1 and lep2, side-by-side.
2. `make_1D_delta_plots` = Differences in kinematics ($\Delta p_{T}, \Delta \eta$, etc.)

#### To Fix:
- If plotting $p_{T}$, make y-label have 'Events/[# GeV]'.
- Consider doing df['branch'].values to increase efficiency.
- Consider not creating so many extra variables to increase efficiency.

In [11]:
import sys
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append('/Users/Jake/')
sys.path.append('/Users/Jake/HiggsMassMeasurement/')
sys.path.append('/Users/Jake/HiggsMassMeasurement/d0_Studies/')

# Neat tricks.
from itertools import chain
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.patches import Rectangle
from IPython.display import display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Local imports. 
from PyUtils.Utils_Files import makeDirs, make_str_title_friendly
from PyUtils.Utils_Plotting import change_cmap_bkg_to_white, save_plots_to_outpath
from PyUtils.Utils_Physics import theta2pseudorap, pseudorap2theta, calc_dR, calc_dphi
from PyUtils.Utils_Statistics import gaussian
from PyUtils.Utils_Collection_Helpers import weave_lists
# from d0_Utils.d0_cls import KinemBinnedEtaPt
from d0_Utils.d0_fns import (add_underoverflow_entries, make_binning_array, shift_binning_array, #make_1D_dist,
                             make_stats_legend_for_1dhist, make_stats_legend_for_2dhist, make_kinem_subplot, get_stats_1Dhist, get_stats_2Dhist)
from d0_Utils.d0_dicts import color_dict, label_LaTeX_dict

plt.rcParams.update({'figure.max_open_warning': 10})    # Related to saving memory and opening plots.

# pd.options.display.max_columns = 23
pd.options.display.max_columns = None

plt.style.use('cmsstyle_plot')

In [2]:
%%time
infile_path_MC_2016 = '/Users/Jake/Desktop/MC_2016.h5'
infile_path_MC_2017 = '/Users/Jake/Desktop/Research/Higgs_Mass_Measurement/NTuples/MC/MC_2017.feather'
infile_path_MC_2018 = '/Users/Jake/Desktop/Research/Higgs_Mass_Measurement/NTuples/MC/MC_2018.feather'

# df_MC_2016 = pd.read_feather(infile_path_MC_2016)
# df_MC_2017 = pd.read_feather(infile_path_MC_2017)
# df_MC_2018 = pd.read_feather(infile_path_MC_2018)

df_MC_2016 = pd.read_hdf(infile_path_MC_2016)
# df_MC_2017 = pd.read_hdf(infile_path_MC_2017)
# df_MC_2018 = pd.read_hdf(infile_path_MC_2018)

CPU times: user 195 ms, sys: 6.76 s, total: 6.95 s
Wall time: 17.7 s


In [ ]:
class KinemBinnedEtaPt():

    def __init__(self, df_original, n_evts, eta_cut_ls, pT_cut_ls, use_ptotal_instead=False, dR_cut=0.02, verbose=False):
        """
        Pass in a DataFrame (DF) and specify the eta and pT cuts to create a subset of DF.
        
        Parameters
        ----------
        df_original : pandas.DataFrame
            ROOT file converted into a DataFrame. Columns are branches. Rows are events.
        n_evts : int
            Number of events to search over - not guaranteed to find this many events! 
            Use '-1' to loop over all events in the df. 
        eta_cut_ls : list or array-like of floats
            A list of [eta_min, eta_max]. Example: [0.9, 1.8]
        pT_cut_ls : list or array-like of floats
            A list of [pT_min, pT_max]. Example: [5, 20]
        dR_cut : float
            A cut to save events in which muon1 and muon2 both have dR < dR_cut.
            
        FIXME:
            - The methods further down are more developed than the methods closer to __init__(). 
                Therefore, clean up and consolidate the earlier methods. 
            - Make method: make_selection(sel)
                Returns only events and all info which pass certain selection criteria (sel).
            - Make plotting methods shorter.
            - In most places, pT could stand for either p or pT (depending on 'use_ptotal_instead').
                Just be careful because not all places are adapted for p!
        """
        if n_evts == -1:
            n_evts = len(df_original)
        df = df_original.loc[:n_evts].copy()    # Original DF. 
        
#         self.binned_df = None    # The DF with the kinematic region cuts applied.
        self.n_evts_asked_for = n_evts
        self.n_evts_found = -999
        
        self.use_ptotal_instead = use_ptotal_instead
        self.p_str = "p" if (self.use_ptotal_instead) else "pT"
        self.p_str_latex = "p" if (self.use_ptotal_instead) else "p_{T}"
        
        self.eta_min   = eta_cut_ls[0]
        self.eta_max   = eta_cut_ls[1]
        self.pT_min    = pT_cut_ls[0] 
        self.pT_max    = pT_cut_ls[1]         
        self.dR_cut    = dR_cut    
        self.massZ_min = 60
        self.massZ_max = 120    
        
        self.apply_pT_eta_cuts(df, verbose)
        self.save_exclusive_masks()
        
    def apply_pT_eta_cuts(self, df, verbose=False):
        """
        Creates a subset of the original DataFrame.
        The subset contains only the events in which either muon1 passes all eta and pT cuts, or muon2 does, or both muons do. 
        """
        # Cuts:
        eta_min   = self.eta_min  
        eta_max   = self.eta_max  
        pT_min    = self.pT_min   
        pT_max    = self.pT_max   
        dR_cut    = self.dR_cut   
        massZ_min = self.massZ_min
        massZ_max = self.massZ_max

        #----------------#
        #--- Analysis ---#
        #----------------#
        # GEN info. 
        eta1_gen_ser = df['genLep_eta1']
        eta2_gen_ser = df['genLep_eta2']
        phi1_gen_ser = df['genLep_phi1']
        phi2_gen_ser = df['genLep_phi2']
        pT1_gen_ser  = df['genLep_pt1']
        pT2_gen_ser  = df['genLep_pt2']

        # RECO info.
        eta1_rec_ser = df['eta1']
        eta2_rec_ser = df['eta2']
        phi1_rec_ser = df['phi1']
        phi2_rec_ser = df['phi2']
        pT1_rec_ser  = df['pT1']
        pT2_rec_ser  = df['pT2']
        
        # Store other variables.
        df.loc[:,'genLep_theta1'] = theta1_gen_ser = pseudorap2theta(eta1_gen_ser)
        df.loc[:,'genLep_theta2'] = theta2_gen_ser = pseudorap2theta(eta2_gen_ser)
        df.loc[:,'theta1'] = theta1_rec_ser = pseudorap2theta(eta1_rec_ser)
        df.loc[:,'theta2'] = theta2_rec_ser = pseudorap2theta(eta2_rec_ser)
        
        df.loc[:,'genLep_p1'] = pT1_gen_ser / np.sin( theta1_gen_ser )
        df.loc[:,'genLep_p2'] = pT2_gen_ser / np.sin( theta2_gen_ser )
        df.loc[:,'p1'] = pT1_rec_ser / np.sin( theta1_rec_ser )  # Total momentum
        df.loc[:,'p2'] = pT2_rec_ser / np.sin( theta2_rec_ser )  # Total momentum
                        
        df.loc[:,'delta_eta1'] = deta1_ser = eta1_rec_ser - eta1_gen_ser
        df.loc[:,'delta_eta2'] = deta2_ser = eta2_rec_ser - eta2_gen_ser
        df.loc[:,'delta_theta1'] = dtheta1_ser = theta1_rec_ser - theta1_gen_ser
        df.loc[:,'delta_theta2'] = dtheta2_ser = theta2_rec_ser - theta2_gen_ser

        # Remember that delta_phi requires special treatment:
        # -pi < delta_phi < pi
        df.loc[:,'delta_phi1'] = dphi1_ser = calc_dphi(phi1_rec_ser, phi1_gen_ser)
        df.loc[:,'delta_phi2'] = dphi2_ser = calc_dphi(phi2_rec_ser, phi2_gen_ser)

        df.loc[:,'delta_R1'] = dR1_ser = calc_dR(deta1_ser, dphi1_ser)
        df.loc[:,'delta_R2'] = dR2_ser = calc_dR(deta2_ser, dphi2_ser)
        df.loc[:,'delta_Rtheta1'] = dRtheta1_ser = calc_dR(dtheta1_ser, dphi1_ser)
        df.loc[:,'delta_Rtheta2'] = dRtheta2_ser = calc_dR(dtheta2_ser, dphi2_ser)
        
        df.loc[:,'delta_pT1'] = dpT1 = pT1_rec_ser - pT1_gen_ser
        df.loc[:,'delta_pT2'] = dpT2 = pT2_rec_ser - pT2_gen_ser
        
        df.loc[:,'delta_pToverpT1'] = dpT1 / pT1_gen_ser
        df.loc[:,'delta_pToverpT2'] = dpT2 / pT2_gen_ser
        
        df.loc[:,'delta_pToverRecpT1'] = dpT1 / pT1_rec_ser
        df.loc[:,'delta_pToverRecpT2'] = dpT2 / pT2_rec_ser     
        
        df.loc[:,'d0BSxq1'] = df['d0BS1'] * df['Id1'].replace(13,-1).replace(-13,1)
        df.loc[:,'d0BSxq2'] = df['d0BS2'] * df['Id2'].replace(13,-1).replace(-13,1)
        df.loc[:,'d0PVxq1'] = df['d0PV1'] * df['Id1'].replace(13,-1).replace(-13,1)
        df.loc[:,'d0PVxq2'] = df['d0PV2'] * df['Id2'].replace(13,-1).replace(-13,1)
        
        # Create masks.
        mask_dR = (dR1_ser < self.dR_cut) & (dR2_ser < self.dR_cut)

        mask_massZ = (massZ_min < df['massZ']) & (df['massZ'] < massZ_max)
        
        if (self.use_ptotal_instead):
            mask_pT1 = (pT_min < df['p1']) & (df['p1'] < pT_max) 
            mask_pT2 = (pT_min < df['p2']) & (df['p2'] < pT_max)
        else:
            mask_pT1 = (pT_min < df['pT1']) & (df['pT1'] < pT_max) 
            mask_pT2 = (pT_min < df['pT2']) & (df['pT2'] < pT_max)
        
        mask_eta1 = (eta_min < abs(df['eta1'])) & (abs(df['eta1']) < eta_max)
        mask_eta2 = (eta_min < abs(df['eta2'])) & (abs(df['eta2']) < eta_max)        

        # Combine masks.
        mask_kinembin_lep1 = mask_eta1 & mask_pT1
        mask_kinembin_lep2 = mask_eta2 & mask_pT2

        all_masks = mask_dR & mask_massZ & (mask_kinembin_lep1 | mask_kinembin_lep2)
        
        # Apply masks and update DataFrame.
        self.binned_df = df[all_masks]
        
        self.cuts =  r"$%d <$ massZ $< %d$ GeV" % (self.massZ_min, self.massZ_max)
        self.cuts += r",   $%.2f < \left| \eta \right| < %.2f$" % (self.eta_min, self.eta_max)
        self.cuts += r",   $%d < %s < %d$" % (self.pT_min, self.p_str_latex, self.pT_max)
        self.cuts += r",   $\Delta R < %.3f$" % (self.dR_cut)
        self.n_evts_found = len(self.binned_df)
        
        if (verbose): 
            perc = self.n_evts_found / float(self.n_evts_asked_for) * 100.
            print(f"Events found: {self.n_evts_found} ({perc:.2f}% of total events), using cuts: {self.cuts}")  
            
    def make2Dplot_pT_vs_eta(self, eta_2D_limits=[-2.5, 2.5, 0.1], pT_2D_limits=[0, 100, 1], save_plot=False, save_as_png=False, verbose=False, outpath=""):
        """
        FIXME: Still need to update this method to work with `use_ptotal_instead` bool. 
        
        Make a 2D plot of pT vs. eta. 
        User can specify the binning along either axis. 
        
        Parameters
        ----------
        eta_2D_limits : list or array-like of floats
            The limits on the horizontal axis.
            [eta_min, eta_max, bin_width]. Example: [-2.5, 2.5, 0.1]
        pT_2D_limits : list or array-like of floats
            The limits on the vertical axis.
            [pT_min, pT_max, bin_width]. Example: [0, 100, 1]
        save_plot : bool
            If True, save the plot as a pdf and png.
        outpath : str
            Path to save plot.
        """
        eta_2D_bins, eta_2D_bin_width = make_binning_array(eta_2D_limits)
        pT_2D_bins, pT_2D_bin_width = make_binning_array(pT_2D_limits)        
        
        fontsize_xlabel = 15
        fontsize_ylabel = 15
        f = plt.figure(figsize=(10,8))

        ax = f.add_subplot()
        ax.set_xlabel(r'$\eta$,  (bin width = %.1f)'            % eta_2D_bin_width, fontsize=fontsize_xlabel)
        ax.set_ylabel(r'p$_{T}$ [GeV],  (bin width = %.1f GeV)' % pT_2D_bin_width,  fontsize=fontsize_ylabel)

        title  = r'$\mu_{1}^{RECO}$ or $\mu_{2}^{RECO}$, in 2016 DY MC' + "\n" + r"cuts: "
        title += r"$%d <$ massZ $< %d$" % (self.massZ_min, self.massZ_max)
        title += r",   $%.1f < \left| \eta \right| < %.1f$" % (self.eta_min, self.eta_max)
        title += r",   $%d < p_{T} < %d$" % (self.pT_min, self.pT_max)
        title += r",   $\Delta R < %.3f$" % (self.dR_cut)
        ax.set_title(title)

        plt.style.use('cmsstyle_plot')

        x_vals = np.append(self.binned_df['eta1'].values, self.binned_df['eta2'].values)
        y_vals = np.append(self.binned_df['pT1'].values,  self.binned_df['pT2'].values)

        newcmp = change_cmap_bkg_to_white('plasma')
        bin_vals, x_bin_edges, y_bin_edges, im = ax.hist2d(x_vals, y_vals, bins=[eta_2D_bins, pT_2D_bins], cmap=newcmp)
        plt.colorbar(im, ax=ax)
        
        file_name  = "pT_vs_eta_2Dplot"
        file_name += "__%.1f_eta_%.1f" % (self.eta_min, self.eta_max)
        file_name += "__%d_pT_%d" % (self.pT_min, self.pT_max)
        
        save_plots_to_outpath(save_plot, outpath, file_name, save_as_png, verbose)
        
    def make2Dplot_dphivsdeta(self, x_2D_limits=[0, 1, 0.1], y_2D_limits=[0, 2, 0.2], save_plot=False, save_as_png=False, verbose=False, outpath=""):
        """
        FIXME: 
            - Still need to update this method to work with `use_ptotal_instead` bool. 
            - Update this method to look like: 
        
        Make a 2D plot of pT vs. eta. 
        ser can specify the binning along either axis. 
        
        Parameters
        ----------
        eta_2D_limits : list or array-like of floats
            The limits on the horizontal axis.
            [eta_min, eta_max, bin_width]. Example: [-2.5, 2.5, 0.1]
        pT_2D_limits : list or array-like of floats
            The limits on the vertical axis.
            [pT_min, pT_max, bin_width]. Example: [0, 100, 1]
        save_plot : bool
            If True, save the plot as a pdf and png.
        outpath : str
            Path to save plot.
            
        FIXME:
        - Maybe generalize this into a method to make any 2D plot.
        """
        
        x_2D_bins, x_2D_bin_width = make_binning_array(x_2D_limits)
        y_2D_bins, y_2D_bin_width = make_binning_array(y_2D_limits)        
        
        f = plt.figure(figsize=(10,8))

        ax = f.add_subplot()
        ax.set_xlabel(r'$\Delta \eta = \eta^{RECO} - \eta^{GEN}$,  (bin width = %.1E)' % x_2D_bin_width, fontsize=15)
        ax.set_ylabel(r'$\Delta \phi = \phi^{RECO} - \phi^{GEN}$,  (bin width = %.1E)' % y_2D_bin_width, fontsize=15)

        title  = r"$\Delta \phi$ vs. $\Delta \eta$, for gen-matched muons in 2016 DY MC, with cuts: " + "\n"
        title += r"$%d <$ massZ $< %d$" % (self.massZ_min, self.massZ_max)
        title += r",   $%.1f < \left| \eta \right| < %.1f$" % (self.eta_min, self.eta_max)
        title += r",   $%d < p_{T} < %d$" % (self.pT_min, self.pT_max)
        title += r",   $\Delta R < %.3f$" % (self.dR_cut)
        ax.set_title(title)

        plt.style.use('cmsstyle_plot')

        x_vals = np.append(self.binned_df['delta_eta1'].values, self.binned_df['delta_eta2'].values)
        y_vals = np.append(self.binned_df['delta_phi1'].values, self.binned_df['delta_phi2'].values)

        newcmp = change_cmap_bkg_to_white('plasma')
        bin_vals, x_bin_edges, y_bin_edges, im = ax.hist2d(x_vals, y_vals, bins=[x_2D_bins, y_2D_bins], cmap=newcmp)
        plt.colorbar(im, ax=ax)
        
        file_name  = "2Dplot_dphi_vs_deta"
        file_name += "__%.1f_eta_%.1f" % (self.eta_min, self.eta_max)
        file_name += "__%d_pT_%d" % (self.pT_min, self.pT_max)
        
        save_plots_to_outpath(save_plot, outpath, file_name, save_as_png, verbose)
                  


    def make2Dplot_dPhi_vs_dEtaANDdTheta(self, 
                                       run_over_only_n_evts=-1, 
                                       x1_bounds=[0, 1, 0.1], 
                                       x2_bounds=[0, 2, 0.2], 
                                       y_bounds=[0, 2, 0.2], 
                                       exclusive=True,
                                       save_plot=False, save_as_png=False, verbose=False, outpath=""):
        """
        Make side-by-side 2D plots of:
            (1) dphi vs. deta  
            (2) dphi vs. dtheta
        User can specify the binning along either axis. 
        This method plots only the muons which pass the selection - not all events in which 
        at least 1 muon pass kinematic bin criteria.
        
        Parameters
        ----------
        eta_2D_limits : list or array-like of floats
            The limits on the horizontal axis.
            [eta_min, eta_max, bin_width]. Example: [-2.5, 2.5, 0.1]
        pT_2D_limits : list or array-like of floats
            The limits on the vertical axis.
            [pT_min, pT_max, bin_width]. Example: [0, 100, 1]
        save_plot : bool
            If True, save the plot as a pdf and png.
        outpath : str
            Path to save plot.
            
        FIXME:
        - Maybe generalize this into a method to make any 2D plot.
        """           
        if run_over_only_n_evts == -1:
            run_over_only_n_evts = self.n_evts_found
        
        x1_2D_bins, x1_2D_bin_width = make_binning_array(x1_bounds)
        x2_2D_bins, x2_2D_bin_width = make_binning_array(x2_bounds)
        y_2D_bins, y_2D_bin_width = make_binning_array(y_bounds)        

        n = run_over_only_n_evts
        
        
        x1_vals, x2_vals_inv, y_vals = self.get_vals_dEta_dTheta_dPhi(run_over_only_n_evts=n, exclusive=exclusive)    
        
        #--- Make plots ---#
        f = plt.figure(figsize=(28, 10))

        # Plot 1: dphi vs. deta
        ax = f.add_subplot(121)
        ax.set_xlabel(r'$\Delta \eta = \eta^{RECO} - \eta^{GEN}$,  (bin width = %.1E)' % x1_2D_bin_width, fontsize=15)
        ax.set_ylabel(r'$\Delta \phi = \phi^{RECO} - \phi^{GEN}$,  (bin width = %.1E)' % y_2D_bin_width, fontsize=15)
        
        title  = r"$\Delta \phi$ vs. $\Delta \eta$, for gen-matched muons in 2016 DY MC"
        title += "\n" + "Only includes muons which pass:" + "\n"
        
        cuts  = r"$%d <$ massZ $< %d$" % (self.massZ_min, self.massZ_max)
        cuts += r",   $%.2f < \left| \eta \right| < %.2f$" % (self.eta_min, self.eta_max)
        cuts += r",   $%d < %s < %d$" % (self.pT_min, self.p_str_latex, self.pT_max)
        cuts += r",   $\Delta R < %.3f$" % (self.dR_cut)
        ax.set_title(title + cuts)
    
        plt.style.use('cmsstyle_plot')

        # Stats: 
        stat_text_x = 0.8
        stat_text_y = 0.9
        
        stats_ls1 = get_stats_2Dhist(x1_vals, y_vals)
        leg_label1 = make_stats_legend_for_2dhist(stats_ls1)
        ax.text(stat_text_x, stat_text_y, leg_label1, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
        
        newcmp = change_cmap_bkg_to_white('rainbow')
        bin_vals, x_bin_edges, y_bin_edges, im = ax.hist2d(x1_vals, y_vals, bins=[x1_2D_bins, y_2D_bins], cmap=newcmp)
        plt.colorbar(im, ax=ax)
        
        # Plot 2: dphi vs. dtheta
        ax = f.add_subplot(122)
        ax.set_xlabel(r'$- \Delta \theta = - (\theta^{RECO} - \theta^{GEN})$,  (bin width = %.1E)' % x2_2D_bin_width, fontsize=15)
        ax.set_ylabel(r'$\Delta \phi = \phi^{RECO} - \phi^{GEN}$,  (bin width = %.1E)' % y_2D_bin_width, fontsize=15)
        title  = r"$\Delta \phi$ vs. $- \Delta \theta$, for gen-matched muons in 2016 DY MC"
        title += "\n" + "Only includes muons which pass:" + "\n"
        ax.set_title(title + cuts)        
        
        plt.style.use('cmsstyle_plot')
            
        # Stats: 
        stats_ls2 = get_stats_2Dhist(x2_vals_inv, y_vals)
        leg_label2 = make_stats_legend_for_2dhist(stats_ls2)
        ax.text(stat_text_x, stat_text_y, leg_label2, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
        
        newcmp = change_cmap_bkg_to_white('rainbow')
        bin_vals, x_bin_edges, y_bin_edges, im = ax.hist2d(x2_vals_inv, y_vals, bins=[x2_2D_bins, y_2D_bins], cmap=newcmp)
        
        plt.colorbar(im, ax=ax)        
        
        # Save plots, if you want. 
        file_name  = "2Dplot_dphi_vs_detaANDdtheta"
        file_name += "__%.2f_eta_%.2f" % (self.eta_min, self.eta_max)
        file_name += "__%d_%s_%d" % (self.pT_min, self.p_str, self.pT_max)
        
        save_plots_to_outpath(save_plot, outpath, file_name, save_as_png, verbose)
        
        # If exclusive cuts: n_muon_entries should be >= number of events (self.n_evts_found).
        # If inclusive cuts: n_muon_entries should be TWICE the number of events (self.n_evts_found). CHECK IT!
        self.n_muon_entries = len(x1_vals)  

    def save_exclusive_masks(self):
        """
        Save two boolean masks:
            mask 1 -> shows which events muon1 passes
            mask 2 -> shows which events muon2 passes
        Keeping the masks separate like this is what is exlusive about the masks. 
        """
#         if run_over_only_n_evts == -1:
#             run_over_only_n_evts = len(self.binned_df)
#         df = self.binned_df[:run_over_only_n_evts]
        # I don't think it's a good idea to slice the DF so early!

        df = self.binned_df
    
        # Create Masks
        if (self.use_ptotal_instead):
            mask_pT1 = (self.pT_min < df['p1']) & (df['p1'] < self.pT_max) 
            mask_pT2 = (self.pT_min < df['p2']) & (df['p2'] < self.pT_max)
        else:
            mask_pT1 = (self.pT_min < df['pT1']) & (df['pT1'] < self.pT_max) 
            mask_pT2 = (self.pT_min < df['pT2']) & (df['pT2'] < self.pT_max)
        
        mask_eta1 = (self.eta_min < abs(df['eta1'])) & (abs(df['eta1']) < self.eta_max)
        mask_eta2 = (self.eta_min < abs(df['eta2'])) & (abs(df['eta2']) < self.eta_max)        

        # Combine masks and save them.
        self.mask_kinembin_lep1 = mask_eta1 & mask_pT1
        self.mask_kinembin_lep2 = mask_eta2 & mask_pT2
        
    def get_vals_dEta_dTheta_dPhi(self, run_over_only_n_evts=-1, exclusive=True):
        """
        Return the muon data which pass selection.

        Wait... I only select the first n muons which pass selection. 
        I first grab all muon1 and then start grabbing muon2...
        If muon1 always has a higher pT than muon2, then this could introduce some bias. 
        THIS IS THE CASE: muon1 has higher pT 99.5% of the time. 
        
        [X] Solution: weave mu1 and mu2 vals together. 
        
        Returns:
        --------
        x1_vals : array 
            DeltaEta1 values (muon1) woven together with DeltaEta2 values (muon2).
            All values satisfy selection criteria for this kinematic bin.
        x2_vals : array 
            Negative DeltaTheta1 values (muon1) woven together with Negative DeltaTheta2 values (muon2).
            All values satisfy selection criteria for this kinematic bin.
        y_vals : array 
            DeltaPhi1 values (muon1) woven together with DeltaPhi2 values (muon2).
            All values satisfy selection criteria for this kinematic bin.
        """
        mask_kinembin_lep1 = self.mask_kinembin_lep1
        mask_kinembin_lep2 = self.mask_kinembin_lep2
        
        df = self.binned_df
        #--- Get data ---#
        if (exclusive):   
            # Keep only the muons which pass the kinem bin selection. 
            deta_vals_muon1 = df['delta_eta1'][mask_kinembin_lep1].values  # Muon 1 passes kinem bin selection.
            deta_vals_muon2 = df['delta_eta2'][mask_kinembin_lep2].values  # Muon 2 passes kinem bin selection.
            
            dtheta_vals_muon1 = df['delta_theta1'][mask_kinembin_lep1].values  # Muon 1 passes kinem bin selection.
            dtheta_vals_muon2 = df['delta_theta2'][mask_kinembin_lep2].values  # Muon 2 passes kinem bin selection.
            
            y_vals_muon1 = df['delta_phi1'][mask_kinembin_lep1].values
            y_vals_muon2 = df['delta_phi2'][mask_kinembin_lep2].values
            
            if run_over_only_n_evts == -1:
                # Running over all events. No need to slice.
                x1_vals = np.append(deta_vals_muon1, deta_vals_muon2)
                x2_vals = np.append(dtheta_vals_muon1, dtheta_vals_muon2)
                y_vals = np.append(y_vals_muon1, y_vals_muon2)
            else:
                n = run_over_only_n_evts
                # Weave muon1 and muon2 values together in systematic way.
                # Otherwise muon1 values would mostly be selected when doing a slice, like [:n].
                x1_weave_ls = weave_lists(deta_vals_muon1, deta_vals_muon2)
                x1_vals = np.array(x1_weave_ls)[:n]

                x2_weave_ls = weave_lists(dtheta_vals_muon1, dtheta_vals_muon2)
                x2_vals = np.array(x2_weave_ls)[:n]
                x2_vals = x2_vals * -1

                y_weave_ls = weave_lists(y_vals_muon1, y_vals_muon2)
                y_vals = np.array(y_weave_ls)[:n]
            
        else:
            #--------# DEPRECATED FOR NOW.
            # Keep both muons from each event in which AT LEAST ONE muon passed the kinem bin selection. 
            raise RuntimeError("Stopping now. This section hasn't been fully developed.\nSet exclusive=True.")
            
            x1_vals = np.append(self.binned_df['delta_eta1'][:n].values, self.binned_df['delta_eta2'][:n].values)
            y_vals = np.append(self.binned_df['delta_phi1'][:n].values, self.binned_df['delta_phi2'][:n].values)

            x2_vals = np.append(self.binned_df['delta_theta1'][:n].values, self.binned_df['delta_theta2'][:n].values)
            x2_vals = x2_vals * -1
            y_vals = np.append(self.binned_df['delta_phi1'][:n].values, self.binned_df['delta_phi2'][:n].values)
            #--------#
            
        self.x1_vals = x1_vals
        self.x2_vals = x2_vals
        self.y_vals = y_vals
        
        return x1_vals, x2_vals, y_vals 

In [50]:
class KinemBinnedEtaPt():

    def __init__(self, df_original, n_evts, eta_cut_ls, pT_cut_ls, use_ptotal_instead=False, dR_cut=0.02, verbose=False):
        """
        Pass in a DataFrame (DF) and specify the eta and pT cuts to create a subset of DF.
        
        Parameters
        ----------
        df_original : pandas.DataFrame
            ROOT file converted into a DataFrame. Columns are branches. Rows are events.
        n_evts : int
            Number of events to search over - not guaranteed to find this many events! 
            Use '-1' to loop over all events in the df. 
        eta_cut_ls : list or array-like of floats
            A list of [eta_min, eta_max]. Example: [0.9, 1.8]
        pT_cut_ls : list or array-like of floats
            A list of [pT_min, pT_max]. Example: [5, 20]
        dR_cut : float
            A cut to save events in which muon1 and muon2 both have dR < dR_cut.
            
        FIXME:
            - The methods further down are more developed than the methods closer to __init__(). 
                Therefore, clean up and consolidate the earlier methods. 
            - Make method: make_selection(sel)
                Returns only events and all info which pass certain selection criteria (sel).
            - Make plotting methods shorter.
            - In most places, pT could stand for either p or pT (depending on 'use_ptotal_instead').
                Just be careful because not all places are adapted for p!
        """
        if n_evts == -1:
            n_evts = len(df_original)
        df = df_original.loc[:n_evts].copy()    # Original DF. 
        
#         self.binned_df = None    # The DF with the kinematic region cuts applied.
        self.n_evts_asked_for = n_evts
        self.n_evts_found = -999
        
        self.use_ptotal_instead = use_ptotal_instead
        self.p_str = "p" if (self.use_ptotal_instead) else "pT"
        self.p_str_latex = "p" if (self.use_ptotal_instead) else "p_{T}"
        
        self.eta_min   = eta_cut_ls[0]
        self.eta_max   = eta_cut_ls[1]
        self.pT_min    = pT_cut_ls[0] 
        self.pT_max    = pT_cut_ls[1]         
        self.dR_cut    = dR_cut    
        self.massZ_min = 60
        self.massZ_max = 120    
        
        self.apply_pT_eta_cuts(df, verbose)
        self.save_exclusive_masks()
        
    def apply_pT_eta_cuts(self, df, verbose=False):
        """
        Creates a subset of the original DataFrame.
        The subset contains only the events in which either muon1 passes all eta and pT cuts, or muon2 does, or both muons do. 
        """
        # Cuts:
        eta_min   = self.eta_min  
        eta_max   = self.eta_max  
        pT_min    = self.pT_min   
        pT_max    = self.pT_max   
        dR_cut    = self.dR_cut   
        massZ_min = self.massZ_min
        massZ_max = self.massZ_max

        #----------------#
        #--- Analysis ---#
        #----------------#
        # GEN info. 
        eta1_gen_ser = df['genLep_eta1']
        eta2_gen_ser = df['genLep_eta2']
        phi1_gen_ser = df['genLep_phi1']
        phi2_gen_ser = df['genLep_phi2']
        pT1_gen_ser  = df['genLep_pt1']
        pT2_gen_ser  = df['genLep_pt2']

        # RECO info.
        eta1_rec_ser = df['eta1']
        eta2_rec_ser = df['eta2']
        phi1_rec_ser = df['phi1']
        phi2_rec_ser = df['phi2']
        pT1_rec_ser  = df['pT1']
        pT2_rec_ser  = df['pT2']
        
        # Store other variables.
        df.loc[:,'genLep_theta1'] = theta1_gen_ser = pseudorap2theta(eta1_gen_ser)
        df.loc[:,'genLep_theta2'] = theta2_gen_ser = pseudorap2theta(eta2_gen_ser)
        df.loc[:,'theta1'] = theta1_rec_ser = pseudorap2theta(eta1_rec_ser)
        df.loc[:,'theta2'] = theta2_rec_ser = pseudorap2theta(eta2_rec_ser)
        
        df.loc[:,'genLep_p1'] = pT1_gen_ser / np.sin( theta1_gen_ser )
        df.loc[:,'genLep_p2'] = pT2_gen_ser / np.sin( theta2_gen_ser )
        df.loc[:,'p1'] = pT1_rec_ser / np.sin( theta1_rec_ser )  # Total momentum
        df.loc[:,'p2'] = pT2_rec_ser / np.sin( theta2_rec_ser )  # Total momentum
                        
        df.loc[:,'delta_eta1'] = deta1_ser = eta1_rec_ser - eta1_gen_ser
        df.loc[:,'delta_eta2'] = deta2_ser = eta2_rec_ser - eta2_gen_ser
        df.loc[:,'delta_theta1'] = dtheta1_ser = theta1_rec_ser - theta1_gen_ser
        df.loc[:,'delta_theta2'] = dtheta2_ser = theta2_rec_ser - theta2_gen_ser

        # Remember that delta_phi requires special treatment:
        # -pi < delta_phi < pi
        df.loc[:,'delta_phi1'] = dphi1_ser = calc_dphi(phi1_rec_ser, phi1_gen_ser)
        df.loc[:,'delta_phi2'] = dphi2_ser = calc_dphi(phi2_rec_ser, phi2_gen_ser)

        df.loc[:,'delta_R1'] = dR1_ser = calc_dR(deta1_ser, dphi1_ser)
        df.loc[:,'delta_R2'] = dR2_ser = calc_dR(deta2_ser, dphi2_ser)
        df.loc[:,'delta_Rtheta1'] = dRtheta1_ser = calc_dR(dtheta1_ser, dphi1_ser)
        df.loc[:,'delta_Rtheta2'] = dRtheta2_ser = calc_dR(dtheta2_ser, dphi2_ser)
        
        df.loc[:,'delta_pT1'] = dpT1 = pT1_rec_ser - pT1_gen_ser
        df.loc[:,'delta_pT2'] = dpT2 = pT2_rec_ser - pT2_gen_ser
        
        df.loc[:,'delta_pToverpT1'] = dpT1 / pT1_gen_ser
        df.loc[:,'delta_pToverpT2'] = dpT2 / pT2_gen_ser
        
        df.loc[:,'delta_pToverRecpT1'] = dpT1 / pT1_rec_ser
        df.loc[:,'delta_pToverRecpT2'] = dpT2 / pT2_rec_ser     
        
        df.loc[:,'d0BSxq1'] = df['d0BS1'] * df['Id1'].replace(13,-1).replace(-13,1)
        df.loc[:,'d0BSxq2'] = df['d0BS2'] * df['Id2'].replace(13,-1).replace(-13,1)
        df.loc[:,'d0PVxq1'] = df['d0PV1'] * df['Id1'].replace(13,-1).replace(-13,1)
        df.loc[:,'d0PVxq2'] = df['d0PV2'] * df['Id2'].replace(13,-1).replace(-13,1)
        
        # Create masks.
        mask_dR = (dR1_ser < self.dR_cut) & (dR2_ser < self.dR_cut)

        mask_massZ = (massZ_min < df['massZ']) & (df['massZ'] < massZ_max)
        
        if (self.use_ptotal_instead):
            mask_pT1 = (pT_min < df['p1']) & (df['p1'] < pT_max) 
            mask_pT2 = (pT_min < df['p2']) & (df['p2'] < pT_max)
        else:
            mask_pT1 = (pT_min < df['pT1']) & (df['pT1'] < pT_max) 
            mask_pT2 = (pT_min < df['pT2']) & (df['pT2'] < pT_max)
        
        mask_eta1 = (eta_min < abs(df['eta1'])) & (abs(df['eta1']) < eta_max)
        mask_eta2 = (eta_min < abs(df['eta2'])) & (abs(df['eta2']) < eta_max)        

        # Combine masks.
        mask_kinembin_lep1 = mask_eta1 & mask_pT1
        mask_kinembin_lep2 = mask_eta2 & mask_pT2

        all_masks = mask_dR & mask_massZ & (mask_kinembin_lep1 | mask_kinembin_lep2)
        
        # Apply masks and update DataFrame.
        self.binned_df = df[all_masks]
        
        self.cuts =  r"$%d <$ massZ $< %d$ GeV" % (self.massZ_min, self.massZ_max)
        self.cuts += r",   $%.2f < \left| \eta \right| < %.2f$" % (self.eta_min, self.eta_max)
        self.cuts += r",   $%d < %s < %d$" % (self.pT_min, self.p_str_latex, self.pT_max)
        self.cuts += r",   $\Delta R < %.3f$" % (self.dR_cut)
        self.n_evts_found = len(self.binned_df)
        
        if (verbose): 
            perc = self.n_evts_found / float(self.n_evts_asked_for) * 100.
            print(f"Events found: {self.n_evts_found} ({perc:.2f}% of total events), using cuts: {self.cuts}")  
            
    def make2Dplot_pT_vs_eta(self, eta_2D_limits=[-2.5, 2.5, 0.1], pT_2D_limits=[0, 100, 1], save_plot=False, save_as_png=False, verbose=False, outpath=""):
        """
        FIXME: Still need to update this method to work with `use_ptotal_instead` bool. 
        
        Make a 2D plot of pT vs. eta. 
        User can specify the binning along either axis. 
        
        Parameters
        ----------
        eta_2D_limits : list or array-like of floats
            The limits on the horizontal axis.
            [eta_min, eta_max, bin_width]. Example: [-2.5, 2.5, 0.1]
        pT_2D_limits : list or array-like of floats
            The limits on the vertical axis.
            [pT_min, pT_max, bin_width]. Example: [0, 100, 1]
        save_plot : bool
            If True, save the plot as a pdf and png.
        outpath : str
            Path to save plot.
        """
        eta_2D_bins, eta_2D_bin_width = make_binning_array(eta_2D_limits)
        pT_2D_bins, pT_2D_bin_width = make_binning_array(pT_2D_limits)        
        
        fontsize_xlabel = 15
        fontsize_ylabel = 15
        f = plt.figure(figsize=(10,8))

        ax = f.add_subplot()
        ax.set_xlabel(r'$\eta$,  (bin width = %.1f)'            % eta_2D_bin_width, fontsize=fontsize_xlabel)
        ax.set_ylabel(r'p$_{T}$ [GeV],  (bin width = %.1f GeV)' % pT_2D_bin_width,  fontsize=fontsize_ylabel)

        title  = r'$\mu_{1}^{RECO}$ or $\mu_{2}^{RECO}$, in 2016 DY MC' + "\n" + r"cuts: "
        title += r"$%d <$ massZ $< %d$" % (self.massZ_min, self.massZ_max)
        title += r",   $%.1f < \left| \eta \right| < %.1f$" % (self.eta_min, self.eta_max)
        title += r",   $%d < p_{T} < %d$" % (self.pT_min, self.pT_max)
        title += r",   $\Delta R < %.3f$" % (self.dR_cut)
        ax.set_title(title)

        plt.style.use('cmsstyle_plot')

        x_vals = np.append(self.binned_df['eta1'].values, self.binned_df['eta2'].values)
        y_vals = np.append(self.binned_df['pT1'].values,  self.binned_df['pT2'].values)

        newcmp = change_cmap_bkg_to_white('plasma')
        bin_vals, x_bin_edges, y_bin_edges, im = ax.hist2d(x_vals, y_vals, bins=[eta_2D_bins, pT_2D_bins], cmap=newcmp)
        plt.colorbar(im, ax=ax)
        
        file_name  = "pT_vs_eta_2Dplot"
        file_name += "__%.1f_eta_%.1f" % (self.eta_min, self.eta_max)
        file_name += "__%d_pT_%d" % (self.pT_min, self.pT_max)
        
        save_plots_to_outpath(save_plot, outpath, file_name, save_as_png, verbose)
        
    def make2Dplot_dphivsdeta(self, x_2D_limits=[0, 1, 0.1], y_2D_limits=[0, 2, 0.2], save_plot=False, save_as_png=False, verbose=False, outpath=""):
        """
        FIXME: 
            - Still need to update this method to work with `use_ptotal_instead` bool. 
            - Update this method to look like: 
        
        Make a 2D plot of pT vs. eta. 
        ser can specify the binning along either axis. 
        
        Parameters
        ----------
        eta_2D_limits : list or array-like of floats
            The limits on the horizontal axis.
            [eta_min, eta_max, bin_width]. Example: [-2.5, 2.5, 0.1]
        pT_2D_limits : list or array-like of floats
            The limits on the vertical axis.
            [pT_min, pT_max, bin_width]. Example: [0, 100, 1]
        save_plot : bool
            If True, save the plot as a pdf and png.
        outpath : str
            Path to save plot.
            
        FIXME:
        - Maybe generalize this into a method to make any 2D plot.
        """
        
        x_2D_bins, x_2D_bin_width = make_binning_array(x_2D_limits)
        y_2D_bins, y_2D_bin_width = make_binning_array(y_2D_limits)        
        
        f = plt.figure(figsize=(10,8))

        ax = f.add_subplot()
        ax.set_xlabel(r'$\Delta \eta = \eta^{RECO} - \eta^{GEN}$,  (bin width = %.1E)' % x_2D_bin_width, fontsize=15)
        ax.set_ylabel(r'$\Delta \phi = \phi^{RECO} - \phi^{GEN}$,  (bin width = %.1E)' % y_2D_bin_width, fontsize=15)

        title  = r"$\Delta \phi$ vs. $\Delta \eta$, for gen-matched muons in 2016 DY MC, with cuts: " + "\n"
        title += r"$%d <$ massZ $< %d$" % (self.massZ_min, self.massZ_max)
        title += r",   $%.1f < \left| \eta \right| < %.1f$" % (self.eta_min, self.eta_max)
        title += r",   $%d < p_{T} < %d$" % (self.pT_min, self.pT_max)
        title += r",   $\Delta R < %.3f$" % (self.dR_cut)
        ax.set_title(title)

        plt.style.use('cmsstyle_plot')

        x_vals = np.append(self.binned_df['delta_eta1'].values, self.binned_df['delta_eta2'].values)
        y_vals = np.append(self.binned_df['delta_phi1'].values, self.binned_df['delta_phi2'].values)

        newcmp = change_cmap_bkg_to_white('plasma')
        bin_vals, x_bin_edges, y_bin_edges, im = ax.hist2d(x_vals, y_vals, bins=[x_2D_bins, y_2D_bins], cmap=newcmp)
        plt.colorbar(im, ax=ax)
        
        file_name  = "2Dplot_dphi_vs_deta"
        file_name += "__%.1f_eta_%.1f" % (self.eta_min, self.eta_max)
        file_name += "__%d_pT_%d" % (self.pT_min, self.pT_max)
        
        save_plots_to_outpath(save_plot, outpath, file_name, save_as_png, verbose)
                  


    def make2Dplot_dPhi_vs_dEtaANDdTheta(self, 
                                       run_over_only_n_evts=-1, 
                                       x1_bounds=[0, 1, 0.1], 
                                       x2_bounds=[0, 2, 0.2], 
                                       y_bounds=[0, 2, 0.2], 
                                       exclusive=True,
                                       save_plot=False, save_as_png=False, verbose=False, outpath=""):
        """
        Make side-by-side 2D plots of:
            (1) dphi vs. deta  
            (2) dphi vs. dtheta
        User can specify the binning along either axis. 
        This method plots only the muons which pass the selection - not all events in which 
        at least 1 muon pass kinematic bin criteria.
        
        Parameters
        ----------
        eta_2D_limits : list or array-like of floats
            The limits on the horizontal axis.
            [eta_min, eta_max, bin_width]. Example: [-2.5, 2.5, 0.1]
        pT_2D_limits : list or array-like of floats
            The limits on the vertical axis.
            [pT_min, pT_max, bin_width]. Example: [0, 100, 1]
        save_plot : bool
            If True, save the plot as a pdf and png.
        outpath : str
            Path to save plot.
            
        FIXME:
        - Maybe generalize this into a method to make any 2D plot.
        """           
        if run_over_only_n_evts == -1:
            run_over_only_n_evts = self.n_evts_found
        
        x1_2D_bins, x1_2D_bin_width = make_binning_array(x1_bounds)
        x2_2D_bins, x2_2D_bin_width = make_binning_array(x2_bounds)
        y_2D_bins, y_2D_bin_width = make_binning_array(y_bounds)        

        n = run_over_only_n_evts
        
        
        x1_vals, x2_vals_inv, y_vals = self.get_vals_dEta_dTheta_dPhi(run_over_only_n_evts=n, exclusive=exclusive)    
        
        #--- Make plots ---#
        f = plt.figure(figsize=(28, 10))

        # Plot 1: dphi vs. deta
        ax = f.add_subplot(121)
        ax.set_xlabel(r'$\Delta \eta = \eta^{RECO} - \eta^{GEN}$,  (bin width = %.1E)' % x1_2D_bin_width, fontsize=15)
        ax.set_ylabel(r'$\Delta \phi = \phi^{RECO} - \phi^{GEN}$,  (bin width = %.1E)' % y_2D_bin_width, fontsize=15)
        
        title  = r"$\Delta \phi$ vs. $\Delta \eta$, for gen-matched muons in 2016 DY MC"
        title += "\n" + "Only includes muons which pass:" + "\n"
        
        cuts  = r"$%d <$ massZ $< %d$" % (self.massZ_min, self.massZ_max)
        cuts += r",   $%.2f < \left| \eta \right| < %.2f$" % (self.eta_min, self.eta_max)
        cuts += r",   $%d < %s < %d$" % (self.pT_min, self.p_str_latex, self.pT_max)
        cuts += r",   $\Delta R < %.3f$" % (self.dR_cut)
        ax.set_title(title + cuts)
    
        plt.style.use('cmsstyle_plot')

        # Stats: 
        stat_text_x = 0.8
        stat_text_y = 0.9
        
        stats_ls1 = get_stats_2Dhist(x1_vals, y_vals)
        leg_label1 = make_stats_legend_for_2dhist(stats_ls1)
        ax.text(stat_text_x, stat_text_y, leg_label1, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
        
        newcmp = change_cmap_bkg_to_white('rainbow')
        bin_vals, x_bin_edges, y_bin_edges, im = ax.hist2d(x1_vals, y_vals, bins=[x1_2D_bins, y_2D_bins], cmap=newcmp)
        plt.colorbar(im, ax=ax)
        
        # Plot 2: dphi vs. dtheta
        ax = f.add_subplot(122)
        ax.set_xlabel(r'$- \Delta \theta = - (\theta^{RECO} - \theta^{GEN})$,  (bin width = %.1E)' % x2_2D_bin_width, fontsize=15)
        ax.set_ylabel(r'$\Delta \phi = \phi^{RECO} - \phi^{GEN}$,  (bin width = %.1E)' % y_2D_bin_width, fontsize=15)
        title  = r"$\Delta \phi$ vs. $- \Delta \theta$, for gen-matched muons in 2016 DY MC"
        title += "\n" + "Only includes muons which pass:" + "\n"
        ax.set_title(title + cuts)        
        
        plt.style.use('cmsstyle_plot')
            
        # Stats: 
        stats_ls2 = get_stats_2Dhist(x2_vals_inv, y_vals)
        leg_label2 = make_stats_legend_for_2dhist(stats_ls2)
        ax.text(stat_text_x, stat_text_y, leg_label2, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
        
        newcmp = change_cmap_bkg_to_white('rainbow')
        bin_vals, x_bin_edges, y_bin_edges, im = ax.hist2d(x2_vals_inv, y_vals, bins=[x2_2D_bins, y_2D_bins], cmap=newcmp)
        
        plt.colorbar(im, ax=ax)        
        
        # Save plots, if you want. 
        file_name  = "2Dplot_dphi_vs_detaANDdtheta"
        file_name += "__%.2f_eta_%.2f" % (self.eta_min, self.eta_max)
        file_name += "__%d_%s_%d" % (self.pT_min, self.p_str, self.pT_max)
        
        save_plots_to_outpath(save_plot, outpath, file_name, save_as_png, verbose)
        
        # If exclusive cuts: n_muon_entries should be >= number of events (self.n_evts_found).
        # If inclusive cuts: n_muon_entries should be TWICE the number of events (self.n_evts_found). CHECK IT!
        self.n_muon_entries = len(x1_vals)  

    def save_exclusive_masks(self):
        """
        Save two boolean masks:
            mask 1 -> shows which events muon1 passes
            mask 2 -> shows which events muon2 passes
        Keeping the masks separate like this is what is exlusive about the masks. 
        """
#         if run_over_only_n_evts == -1:
#             run_over_only_n_evts = len(self.binned_df)
#         df = self.binned_df[:run_over_only_n_evts]
        # I don't think it's a good idea to slice the DF so early!

        df = self.binned_df
    
        # Create Masks
        if (self.use_ptotal_instead):
            mask_pT1 = (self.pT_min < df['p1']) & (df['p1'] < self.pT_max) 
            mask_pT2 = (self.pT_min < df['p2']) & (df['p2'] < self.pT_max)
        else:
            mask_pT1 = (self.pT_min < df['pT1']) & (df['pT1'] < self.pT_max) 
            mask_pT2 = (self.pT_min < df['pT2']) & (df['pT2'] < self.pT_max)
        
        mask_eta1 = (self.eta_min < abs(df['eta1'])) & (abs(df['eta1']) < self.eta_max)
        mask_eta2 = (self.eta_min < abs(df['eta2'])) & (abs(df['eta2']) < self.eta_max)        

        # Combine masks and save them.
        self.mask_kinembin_lep1 = mask_eta1 & mask_pT1
        self.mask_kinembin_lep2 = mask_eta2 & mask_pT2
        
    def get_vals_dEta_dTheta_dPhi(self, run_over_only_n_evts=-1, exclusive=True):
        """
        Return the muon data which pass selection.

        Wait... I only select the first n muons which pass selection. 
        I first grab all muon1 and then start grabbing muon2...
        If muon1 always has a higher pT than muon2, then this could introduce some bias. 
        THIS IS THE CASE: muon1 has higher pT 99.5% of the time. 
        
        [X] Solution: weave mu1 and mu2 vals together. 
        
        Returns:
        --------
        x1_vals : array 
            DeltaEta1 values (muon1) woven together with DeltaEta2 values (muon2).
            All values satisfy selection criteria for this kinematic bin.
        x2_vals : array 
            Negative DeltaTheta1 values (muon1) woven together with Negative DeltaTheta2 values (muon2).
            All values satisfy selection criteria for this kinematic bin.
        y_vals : array 
            DeltaPhi1 values (muon1) woven together with DeltaPhi2 values (muon2).
            All values satisfy selection criteria for this kinematic bin.
        """
        mask_kinembin_lep1 = self.mask_kinembin_lep1
        mask_kinembin_lep2 = self.mask_kinembin_lep2
        
        df = self.binned_df
        #--- Get data ---#
        if (exclusive):   
            # Keep only the muons which pass the kinem bin selection. 
            deta_vals_muon1 = df['delta_eta1'][mask_kinembin_lep1].values  # Muon 1 passes kinem bin selection.
            deta_vals_muon2 = df['delta_eta2'][mask_kinembin_lep2].values  # Muon 2 passes kinem bin selection.
            
            dtheta_vals_muon1 = df['delta_theta1'][mask_kinembin_lep1].values  # Muon 1 passes kinem bin selection.
            dtheta_vals_muon2 = df['delta_theta2'][mask_kinembin_lep2].values  # Muon 2 passes kinem bin selection.
            
            y_vals_muon1 = df['delta_phi1'][mask_kinembin_lep1].values
            y_vals_muon2 = df['delta_phi2'][mask_kinembin_lep2].values
            
            if run_over_only_n_evts == -1:
                # Running over all events. No need to slice.
                x1_vals = np.append(deta_vals_muon1, deta_vals_muon2)
                x2_vals = np.append(dtheta_vals_muon1, dtheta_vals_muon2)
                y_vals = np.append(y_vals_muon1, y_vals_muon2)
            else:
                n = run_over_only_n_evts
                # Weave muon1 and muon2 values together in systematic way.
                # Otherwise muon1 values would mostly be selected when doing a slice, like [:n].
                x1_weave_ls = weave_lists(deta_vals_muon1, deta_vals_muon2)
                x1_vals = np.array(x1_weave_ls)[:n]

                x2_weave_ls = weave_lists(dtheta_vals_muon1, dtheta_vals_muon2)
                x2_vals = np.array(x2_weave_ls)[:n]
                x2_vals = x2_vals * -1

                y_weave_ls = weave_lists(y_vals_muon1, y_vals_muon2)
                y_vals = np.array(y_weave_ls)[:n]
            
        else:
            #--------# DEPRECATED FOR NOW.
            # Keep both muons from each event in which AT LEAST ONE muon passed the kinem bin selection. 
            raise RuntimeError("Stopping now. This section hasn't been fully developed.\nSet exclusive=True.")
            
            x1_vals = np.append(self.binned_df['delta_eta1'][:n].values, self.binned_df['delta_eta2'][:n].values)
            y_vals = np.append(self.binned_df['delta_phi1'][:n].values, self.binned_df['delta_phi2'][:n].values)

            x2_vals = np.append(self.binned_df['delta_theta1'][:n].values, self.binned_df['delta_theta2'][:n].values)
            x2_vals = x2_vals * -1
            y_vals = np.append(self.binned_df['delta_phi1'][:n].values, self.binned_df['delta_phi2'][:n].values)
            #--------#
            
        self.x1_vals = x1_vals
        self.x2_vals = x2_vals
        self.y_vals = y_vals
        
        return x1_vals, x2_vals, y_vals 

    def plot_kinem_genrec_comparison(self,
                              kinem_gen, kinem_rec, 
                              x_range_ls=[-1, 1],
                              bin_limits=[-0.5,0.5,0.5], 
                              ax=None, ax_ratio=None, y_max=-1, log_scale=False
                              ):
            """
            FIXME: Need to implement under/overflow bins!

            Plots differences in kinematics. Also makes a ratio plot. 
            """

            df = self.binned_df

            x_bin_arr, x_bin_width = make_binning_array(bin_limits)
            x_bin_centers_arr = shift_binning_array(x_bin_arr)

#             kinem_pT_list = ['genLep_pt1','genLep_pt2','pT1','pT2']
#             kinem_eta_list = ['genLep_eta1','genLep_eta2','eta1','eta2']
#             kinem_phi_list = ['genLep_phi1','genLep_phi2','phi1','phi2']

            lep = kinem_gen[-1]  # Get last character of kinematic. Example: pT1 --> 1. Should be a string!
            
            #--- Get data ---#
            if ("1" in kinem_gen) and ("1" in kinem_rec): 
                data_rec = df[kinem_rec][self.mask_kinembin_lep1].values  # Muon 1 passes kinem bin selection.
                data_gen = df[kinem_gen][self.mask_kinembin_lep1].values  # Muon 1 passes kinem bin selection.
            elif ("2" in kinem_gen) and ("2" in kinem_rec):
                data_rec = df[kinem_rec][self.mask_kinembin_lep2].values  # Muon 1 passes kinem bin selection.
                data_gen = df[kinem_gen][self.mask_kinembin_lep2].values  # Muon 1 passes kinem bin selection.
            else:
                err_msg = "\n    Either kinem_gen or kinem_rec does not end with a '1' or '2', or they are not the same as each other.\nStopping now."
                raise ValueError(err_msg)

            # Gen and Reco stats:
            stats_ls_gen = get_stats_1Dhist(data_gen)
            stats_ls_rec = get_stats_1Dhist(data_rec)

            #----------------#
            #--- Plot It. ---#
            #----------------#
            if (ax is None) or (ax_ratio is None):
                fig = plt.figure(figsize=(10,8))
                ax = fig.add_axes([0.17,0.33,0.825,0.54])  # [low_left_corner_x, low_left_corner_y, width, height]
                ax_ratio = fig.add_axes([0.17,0.12,0.825,0.20])

            leg_label_gen = make_stats_legend_for_1dhist(stats_ls_gen)
            leg_label_rec = make_stats_legend_for_1dhist(stats_ls_rec)

            leg_label_gen = leg_label_gen.replace("Total", "GEN")
            leg_label_rec = leg_label_rec.replace("Total", "REC")

            hist_bin_vals_gen, bin_edges_gen, _ = ax.hist(data_gen, bins=x_bin_arr, histtype='step', color='g', lw=2, label=leg_label_gen)
            hist_bin_vals_rec, bin_edges_rec, _ = ax.hist(data_rec, bins=x_bin_arr, histtype='step', color='r', label=leg_label_rec)

            hist_bin_vals_gen_modified = hist_bin_vals_gen.copy()
            hist_bin_vals_gen_modified[hist_bin_vals_gen == 0] = 0.0000000001
            ratio_vals = (hist_bin_vals_rec - hist_bin_vals_gen) / hist_bin_vals_gen_modified

            ax_ratio.errorbar(x_bin_centers_arr, ratio_vals, xerr=x_bin_width/2, color='black', ms=0, capsize=0, mew=0, ecolor='black', drawstyle='steps-mid', alpha=1)

            # Pretty up the plot. 
            ax_ratio.grid(which='major',axis='x')
            ax_ratio.grid(which='major',axis='y', ls='-')
            ax_ratio.grid(which='minor',axis='y')

            # Hide first tick label on y-axis, since it overlaps with ratio plot's tick label.
            a=ax.get_yticks().tolist()
            a[0]=''
            ax.set_yticklabels(a)

            # Hide main plot's x tick labels.
            plt.setp(ax.get_xticklabels(), visible=False)

            # Only show a few of the tick labels on ratio plot.
            n_tick_labels = 5
            ax_ratio.yaxis.set_major_locator(plt.MaxNLocator(n_tick_labels))
            ax_ratio.axhline(c='r', lw=2, ls='-')

            textsize_legend = 10
            textsize_axislabels = 12
            textsize_title = 12

            unit_gen = label_LaTeX_dict[kinem_gen]["units"]
            unit_rec = label_LaTeX_dict[kinem_rec]["units"]
            
            y_label = hist_y_label(x_bin_width, unit_gen)
            
            # Remember that ax shouldn't have an x-label; it's covered by the ax_ratio. 
            ax.set_ylabel(y_label, fontsize=textsize_axislabels)
            ax.set_title(r"Selection: {}".format(self.cuts), fontsize=textsize_title)

            plt.minorticks_on()

            x_min = x_range_ls[0]
            x_max = x_range_ls[1]
            ax.set_xlim([x_min, x_max])
            ax_ratio.set_xlim([x_min, x_max])
            ax_ratio.set_ylim([-0.12, 0.12])

            x_label_gen = label_LaTeX_dict[kinem_gen]["label"]
            x_label_rec = label_LaTeX_dict[kinem_rec]["label"]
            
            x_label = r"{}, {}".format(x_label_gen, x_label_rec)
            if len(unit_gen) > 0:
                x_label += " [{}]".format(unit_gen)
            ax_ratio.set_xlabel(x_label, fontsize=textsize_axislabels)
            ax_ratio.set_ylabel(r'$\frac{ (\mathrm{REC} - \mathrm{GEN}) }{ \mathrm{GEN} }$', fontsize=textsize_axislabels*1.5)

            y_max = max(max(hist_bin_vals_gen), max(hist_bin_vals_rec))
            ax.set_ylim([0, y_max*1.4])

            ax.legend(loc='upper right', framealpha=0.9, fontsize=textsize_legend)#, horizontalalignment='right')
            
            return ax, ax_ratio
    

    def plot_1D_kinematics(self, lep, kinem, x_limits=[0,0], bin_limits=[0,0,0], ax=None, x_label=r"", y_label="", title="", y_max=-1, log_scale=False, iter_gaus=(False, 0)):
        """
        FIXME:
            [ ] Update docstring.
            [ ] Be able to include any kinematical, like massZ
            
            lep : int
        Either `1` or `2`. Indicates which lepton you are referring to. 
        
        Make a single 1D distribution of some kinematical variable?
        OR
        Plot the dR, dphi, deta, dtheta distributions within this kinematic region?
        """
        df = self.binned_df
        
        if ax is None:
            # Axes doesn't exist yet. Make it.
            fig, ax = plt.subplots(figsize=(10,8))
            
        if bin_limits == [0,0,0]:
            # No bin limits specified, so use default binning for this kinematical variable.
            bin_limits = label_LaTeX_dict[kinem]["default_bin_limits"]
             
        if x_limits == [0,0]:
            # No x-limits specified, so use default x-limits for this kinematical variable.
            x_limits = label_LaTeX_dict[kinem]["default_x_limits"]
            
        unit = label_LaTeX_dict[kinem]["units"]
        
        #--- Get data ---#
        if lep == 1: 
            data = df[kinem][self.mask_kinembin_lep1].values  # Muon 1 passes kinem bin selection.
        elif lep == 2:
            data = df[kinem][self.mask_kinembin_lep2].values  # Muon 1 passes kinem bin selection.
        else:
            err_msg = "\n    You  must specify lep = 1 or 2.\nStopping now."
            raise ValueError(err_msg)
            
        x_bins, binw = make_binning_array(bin_limits)

        if len(x_label) == 0:
            x_label = label_LaTeX_dict[kinem]["label"]
            if len(unit) > 0:
                x_label += " [{}]".format(unit)
            
        if len(y_label) == 0:
            # User didn't specify label, so make it.
            y_label = hist_y_label(binw, unit)
            
        if len(title) == 0:
            title = "Selection: " + self.cuts
        
        ax, bin_vals, bin_edges, stats = make_1D_dist(ax, data, x_limits, x_bins, x_label, y_label, title, y_max=-1, log_scale=False)    
    
        if (iter_gaus[0]):
            # Do iterative fitting procedure.
            iterations = iter_gaus[1]
            msg = "Performing {} iterative Gaussian fits".format(iterations)
            if (iterations == 1):
                msg = msg.replace("fits", "fit") 
            print(msg)

            bin_centers = shift_binning_array(bin_edges)
            
            count = 0
            popt = np.zeros(3)
            while count < iterations:
                count += 1
                
#                 fit_ls = iterative_fit_gaus(bin_centers, bin_vals, fit_range_start=[0,0], iterations=1)
                if count == 1:
                    # First fit: use original histogram's mean and stdev to choose a fit range.
                    this_mean  = stats[1]
                    this_stdev = stats[3]
                else:
                    # Otherwise use the last fit's optimized parameters.
                    this_mean  = popt[1]
                    this_stdev = popt[2]
                this_x_min = this_mean - 2*this_stdev
                this_x_max = this_mean + 2*this_stdev
                
                mask = get_subset_mask(bin_centers, x_min=this_x_min, x_max=this_x_max)
                
                new_bin_centers = bin_centers[mask]
                new_bin_vals = bin_vals[mask]
                
                popt, popt_err, pcov = fit_with_gaussian(new_bin_centers, new_bin_vals, guess_params=[1,this_mean,this_stdev])
                
                # Get the y-vals of the Gaussian fit for plotting
                gaus_y_vals = gaussian(new_bin_centers, *popt)

                leg_label_fit = make_stats_legend_for_gaus_fit(popt, popt_err)
                leg_label_fit = leg_label_fit.replace("Fit", "Fit {}:".format(count))

                ax.plot(new_bin_centers, gaus_y_vals, color=color_dict[count+1], label=leg_label_fit, linestyle='-', marker="")
                ax.legend()
            
            
# def iterative_fit_gaus(x_vals, y_vals, fit_range_start=[0,0], iterations=1):
#     count = 0
#     iter_ls = []
    
#     if fit_range_start == [0,0]:
#         # Use entire x_vals range as fit range.
#         fit_range_new = [min(x_vals), max(x_vals)]
#     else:
#         fit_range_new = fit_range_start
    
#     while count < iterations:
#         count += 1
        
#         popt, popt_err, pcov = fit_with_gaussian(x_vals, y_vals, fit_range=fit_range_new)
#         iter_ls.append( {"popt":popt, "popt_err":popt_err, "pcov":pcov})
        
#         mu = popt[1]
#         sigma = popt[2]
        
#         fit_range_new = [mu - 2*sigma, mu + 2*sigma]
    
#     return iter_ls
        
        
    
#             ax.text(x, y, "Some words", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
# - Using `transform=ax.transAxes` puts x,y in terms of axes coordinates (as opposed to data coordinates)

    def count_in_pT_eta_region_exclusive(self):
        """
        Return the number of muons in the DF subset which pass the eta and pT cuts. 
        'Exclusive' because only those muons which pass such cuts are counted 
        (as opposed to counting both muons from event just because one pass the cuts).
        """
        pass
#         return n_passed = 

In [4]:
from scipy.optimize import curve_fit

def make_stats_legend_for_gaus_fit(popt, popt_err):
    """
    Create a legend label that displays the statistics after fitting a 1D histogram with a Gaussian curve.
    
    Parameters
    ----------
    
    Returns
    -------
    leg_label : str
        A string of the fit statistics, useful for making a legend label. 
    """
    coeff = popt[0]
    mean = popt[1]
    stdev = popt[2]
    
    coeff_err = popt_err[0]
    mean_err = popt_err[1]
    stdev_err = popt_err[2]
    
    leg_label  = r"Fit $C$ = {:4.3E} $\pm$ {:4.3E}".format(coeff, coeff_err) + "\n"
    leg_label += r"Fit $\mu$ = {:4.3E} $\pm$ {:4.3E}".format(mean, mean_err) + "\n"
    leg_label += r"Fit $\sigma$ = {:4.3E} $\pm$ {:4.3E}".format(stdev, stdev_err)

    return leg_label


# def get_subset(x_vals, minmax_tup=(0,0)):
#     """
#     FIXME: POSSIBLY DELETE THIS FUNCTION IN FAVOR OF get_subset_mask().
    
#     Select a subset of contiguous values of an array. 
#     Useful for selecting fit ranges along the 'x-axis'.
    
#     Parameters
#     ----------
#     x_vals : array
#         The original array from which the subset will be made.
#     minmax_tup : array
#         A 2-tuple that contains the min and max values of the desired subset: (x_min, x_max)
        
#     Returns
#     -------
#     x_vals_adj : array
#         The subset of x_vals that begins with x_min and ends with x_max: [x_min, ... , x_max]
#     mask : bool array
#         An array of booleans. If an element is 'True' then the corresponding element in x_vals 
#         is retained in x_vals_adj.
#     """
#     # Select only certain 
#     if fit_range == [0,0]:
#         x_vals_adj = x_vals
        
#     else:  
#         x_min = minmax_tup[0]
#         x_max = minmax_tup[1]

#         mask = (x_min <= x_vals) & (x_vals <= x_max)
#         x_vals_adj = x_vals[mask]
    
#     return x_vals_adj, mask

def get_subset_mask(x_vals, x_min, x_max):
    """
    Return a mask of an array such that: x_min <= element <= x_max
    Useful for selecting fit ranges along the 'x-axis'.
    
    Apply the mask: x_vals[mask]
    
    Parameters
    ----------
    x_vals : array
        The original array from which the subset will be made.
    x_min : float
        The minimum value in x_vals to be kept.
    x_max : float
        The maximum value in x_vals to be kept.
        
    Returns
    -------
    mask : bool array
        An array of booleans. If an element is 'True' then: x_min <= element <= x_max.
        The mask will be the same length as x_vals. 
    """
    mask = (x_min <= x_vals) & (x_vals <= x_max)
    
    return mask


def fit_with_gaussian(x_vals, y_vals, guess_params=[1,1,1]):
    """
    Fit a Gaussian curve to a set of data. Returns the best (mu, sigma) which fit the data.
    
    Parameters
    ----------
    x_vals : array-like
        The x values of the data. 
    y_vals : array-like
        The y values of the data.
    guess_params : array-like
        Initial guess for the parameters of the fitted Gaussian.: [coeff, mean, sigma].
        Putting guess values can speed up fit time and can make fits converge where they would otherwise fail.
        
    Returns
    -------
    popt : 3-element array
        A 3-element array of the the optimized parameters (coefficient, mean, sigma) of the fitted Gaussian: [coeff_best, mu_best, sigma_best]
        From the scipy.optimize.curve_fit docstring:
            "Optimal values for the parameters so that the sum of the squared residuals of f(xdata, *popt) - ydata is minimized."
    pcov : 2d array
        The covariance of popt.
        From the scipy.optimize.curve_fit docstring:
            "To compute one standard deviation errors on the parameters use perr = np.sqrt(np.diag(pcov))"
    """
    popt, pcov = curve_fit(gaussian, x_vals, y_vals, p0=guess_params)
    
    # FIXME: For some strange reason, sigma can turn out to be negative...
    popt[2] = np.abs(popt[2])
    
    popt_err = np.sqrt(np.diag(pcov))

    return popt, popt_err, pcov

def make_1D_dist(ax, data, x_limits, x_bins, x_label, y_label, title, y_max=-1, log_scale=False):
    """
    Draw a kinematic distribution (e.g. eta1, gen_phi2, etc.) to an axes object in a figure.
    
    Parameters
    ----------
    ax : axes object
        The axes to which plot will be drawn.
    data : array-like
        Data to be histogrammed. 
    x_limits : 2-element list
        A list of the x-axis range to be plotted: 
        x_limits=[x-min, x-max]
    x_bins : array-like 
        Array of bin edges. Should be of length = len(n_bins)+1.
    x_label : str
        Label for x-axis.
    y_label : str
        Label for y-axis.
    title : str
        Label for title.
    y_max : float
        Max on y-axis. If y_max <= 0, then matplotlib will choose y_max.
    log_scale : bool
        If True, sets the y-axis to log scale. 
    """
    textsize_legend = 9
    textsize_axislabels = 12
    textsize_title = 12
            
    ax.set_xlabel(x_label, fontsize=textsize_axislabels)
    ax.set_ylabel(y_label, fontsize=textsize_axislabels)
    ax.set_title(title, fontsize=textsize_title)
    
    ax.set_xlim(x_limits)
    ax.grid(False)
    
    mod_data, n_underflow, n_overflow = add_underoverflow_entries(data, x_limits[0], x_limits[1])
    
    if y_max > 0: ax.set_ylim([0,y_max])
    if (log_scale): ax.set_yscale('log')
        
    stats = get_stats_1Dhist(data)
    label_legend = make_stats_legend_for_1dhist(stats)
    bin_vals, bin_edges, _ = ax.hist(mod_data, bins=x_bins, label=label_legend, histtype='step', color='b')
    ax.legend(loc='upper right', framealpha=0.9, fontsize=textsize_legend)
    
    return ax, bin_vals, bin_edges, stats

In [5]:
def hist_y_label(bin_width, unit):
    """
    Return a y-label for a histogram: "Events / [bin_width units]".

    Parameters
    ----------
    bin_width : float
        Bin width of histogram. 
    unit : str
        The unit of whatever distribution you are making.
        
    Returns
    -------
    y_label : str
        The y-label for the distribution.
    """
    # Automatically label y-axis.
    if bin_width < 0.01:
        y_label = "Events / [%.2E]" % bin_width
    else: 
        y_label = "Events / [%.2f]" % bin_width

    if len(unit) > 0:
        y_label = y_label.replace("]", " " + unit + "]")

    return y_label

def make_2by2_subplots_for_ratioplots(fig_width=28, fig_height=16):
    """
    Return a figure, 4 main axes, and 4 ratio axes, useful for making ratio plots. 
    The returned tuples can be intuitively sliced to get the axes you want:
        E.g. ax_ratio_tup[0][1] -> ax12_ratio
        
    Parameters
    ----------
    fig_width : float
        Width of figure.
    fig_height : float
        Height of figure.
        
    Returns
    -------
    fig : figure
        A figure object with 8 axes objects, but 4 main areas. 
        Kind of like subplot(22), but with more control.
    ax_tup : tuple
        A tuple of 4 axes objects. These are the "main" axes for plotting.
        Each main axes object (e.g. ax21) has an associated ratio axes (e.g. ax21_ratio) associated with it. 
    ax_ratio_tup : tuple
        A tuple of 4 axes objects. These are the ratio plot axes. 
        The ratio axes make it useful to make ratio plots of the corresponding main axes object. 
    """
    fig = plt.figure(figsize=(fig_width, fig_height))

    x_left = 0.04
    y_bottom = 0.07

    leftright_spacing = 0.04  # Spacing between plots.
    updown_spacing = 0.06  # Spacing between plots.
    
    buffer_on_right = 0.03
    buffer_on_top = 0.03
    
    magnification = 2.5  # How many times taller the main plot is relative to ratio plot.
    plot_width = (1 - x_left - leftright_spacing - buffer_on_right) * 0.5
    plot_ratio_height = (1 - y_bottom - updown_spacing - buffer_on_top) / (2 * (magnification + 1))
    plot_height = magnification * plot_ratio_height

    # You can derive these formulae pretty easily. Just make a diagram.
    x_right = x_left + plot_width + leftright_spacing
    y_middle = y_bottom + plot_ratio_height + plot_height + updown_spacing

    ax11       = fig.add_axes([x_left, y_middle+plot_ratio_height, plot_width, plot_height])  # [low_left_corner_x, low_left_corner_y, plot_width, height]
    ax11_ratio = fig.add_axes([x_left, y_middle,                   plot_width, plot_ratio_height])

    ax12       = fig.add_axes([x_right, y_middle+plot_ratio_height, plot_width, plot_height])  # [low_left_corner_x, low_left_corner_y, plot_width, height]
    ax12_ratio = fig.add_axes([x_right, y_middle,                   plot_width, plot_ratio_height])

    ax21       = fig.add_axes([x_left, y_bottom+plot_ratio_height, plot_width, plot_height])  # [low_left_corner_x, low_left_corner_y, plot_width, height]
    ax21_ratio = fig.add_axes([x_left, y_bottom,                   plot_width, plot_ratio_height])

    ax22       = fig.add_axes([x_right, y_bottom+plot_ratio_height, plot_width, plot_height])  # [low_left_corner_x, low_left_corner_y, plot_width, height]
    ax22_ratio = fig.add_axes([x_right, y_bottom,                   plot_width, plot_ratio_height])
    
    ax_tup = ((ax11, ax12), (ax21, ax22))
    ax_ratio_tup = ((ax11_ratio, ax12_ratio), (ax21_ratio, ax22_ratio))
    
    return fig, ax_tup, ax_ratio_tup

# Testing:

In [39]:
%%time
%config InlineBackend.figure_format ='retina'

kbin = KinemBinnedEtaPt(df_MC_2016, n_evts=100000, eta_cut_ls=[2.2, 2.4], pT_cut_ls=[5, 100], dR_cut=0.02)

CPU times: user 2.76 s, sys: 1.69 s, total: 4.44 s
Wall time: 2.66 s


# Make all kinematic plots into 1 PDF with iterative fits.

In [38]:
label_LaTeX_dict = {
    "pT1"  : {"label":r"$p_{T1}^{\mathrm{REC}}$", "units":"GeV"},
    "pT2"  : {"label":r"$p_{T2}^{\mathrm{REC}}$", "units":"GeV"},
    
    'eta1' : {"label":r"$\eta_{1}^{\mathrm{REC}}$", "units":""},
    'eta2' : {"label":r"$\eta_{2}^{\mathrm{REC}}$", "units":""},
    
    'theta1' : {"label":r"$\theta_{1}^{\mathrm{REC}}$", "units":""},
    'theta2' : {"label":r"$\theta_{2}^{\mathrm{REC}}$", "units":""},

    'phi1' : {"label":r"$\phi_{1}^{\mathrm{REC}}$", "units":""},
    'phi2' : {"label":r"$\phi_{2}^{\mathrm{REC}}$", "units":""},
    
    'genLep_pt1' : {"label":r"$p_{T1}^{\mathrm{GEN}}$", "units":"GeV"},
    'genLep_pt2' : {"label":r"$p_{T2}^{\mathrm{GEN}}$", "units":"GeV"},
    
    'genLep_eta1' : {"label":r"$\eta_{1}^{\mathrm{GEN}}$", "units":""},
    'genLep_eta2' : {"label":r"$\eta_{2}^{\mathrm{GEN}}$", "units":""},
    
    'genLep_phi1' : {"label":r"$\phi_{1}^{\mathrm{GEN}}$", "units":""},
    'genLep_phi2' : {"label":r"$\phi_{2}^{\mathrm{GEN}}$", "units":""},
    
    'genLep_theta1' : {"label":r"$\theta_{1}^{\mathrm{GEN}}$", "units":""},
    'genLep_theta2' : {"label":r"$\theta_{2}^{\mathrm{GEN}}$", "units":""},
    
    "delta_pT1" : {"label":r"$\Delta p_{T1} \equiv p_{T1}^{\mathrm{REC}} - p_{T1}^{\mathrm{GEN}} $", "units":"GeV"},
    "delta_pT2" : {"label":r"$\Delta p_{T2} \equiv p_{T2}^{\mathrm{REC}} - p_{T2}^{\mathrm{GEN}} $", "units":"GeV"},
    "delta_pToverpT1" : {"label":r"$ \Delta p_{T1} \ / p_{T1}^{\mathrm{GEN}}$", "units":""},
    "delta_pToverpT2" : {"label":r"$ \Delta p_{T2} \ / p_{T2}^{\mathrm{GEN}}$", "units":""},
    
    "delta_pToverRecpT1" : {"label":r"$ \Delta p_{T1} \ / p_{T1}^{\mathrm{REC}}$", "units":""},
    "delta_pToverRecpT2" : {"label":r"$ \Delta p_{T2} \ / p_{T2}^{\mathrm{REC}}$", "units":""},
    
#     "delta_pToverpT2_squared" : {"label":r"$( p_{T}^{\mathrm{REC}} - p_{T}^{\mathrm{GEN}} ) / (p_{T}^{\mathrm{GEN}})^2 $ GeV$^{-1}$", "units":r"GeV$^{-1}$"},
    
    "delta_R1" : {"label":r"$\Delta R_{1} = \sqrt{  (\Delta \eta_{1})^2 + (\Delta \phi_{1})^2   }$", "units":""},
    "delta_R2" : {"label":r"$\Delta R_{2} = \sqrt{  (\Delta \eta_{2})^2 + (\Delta \phi_{2})^2   }$", "units":""},
    
    "delta_eta1" : {"label":r"$\Delta \eta_{1} = \eta_{1}^{\mathrm{REC}} - \eta_{1}^{\mathrm{GEN}}$", "units":""},
    "delta_eta2" : {"label":r"$\Delta \eta_{2} = \eta_{2}^{\mathrm{REC}} - \eta_{2}^{\mathrm{GEN}}$", "units":""},
    
    "delta_theta1" : {"label":r"$\Delta \theta_{1} = \theta_{1}^{\mathrm{REC}} - \theta_{1}^{\mathrm{GEN}}$", "units":""},
    "delta_theta2" : {"label":r"$\Delta \theta_{2} = \theta_{2}^{\mathrm{REC}} - \theta_{2}^{\mathrm{GEN}}$", "units":""},
    
    "delta_phi1" : {"label":r"$\Delta \phi_{1} = \phi_{1}^{\mathrm{REC}} - \phi_{1}^{\mathrm{GEN}}$", "units":""},
    "delta_phi2" : {"label":r"$\Delta \phi_{2} = \phi_{2}^{\mathrm{REC}} - \phi_{2}^{\mathrm{GEN}}$", "units":""},
    
    "d0BS1" : {"label":r"$d_{0, \mathrm{lep1}}^{ \mathrm{BS} }$", "units":"cm"},
    "d0BS2" : {"label":r"$d_{0, \mathrm{lep2}}^{ \mathrm{BS} }$", "units":"cm"},
    "d0PV1" : {"label":r"$d_{0, \mathrm{lep1}}^{ \mathrm{PV} }$", "units":"cm"},
    "d0PV2" : {"label":r"$d_{0, \mathrm{lep2}}^{ \mathrm{PV} }$", "units":"cm"},
    
    "d0BSxq1" : {"label":r"$d_{0}^{ \mathrm{BS} } * \mathrm{charge}(\mu_{1}^{REC})$", "units":"cm"},
    "d0BSxq2" : {"label":r"$d_{0}^{ \mathrm{BS} } * \mathrm{charge}(\mu_{2}^{REC})$", "units":"cm"},
    "d0PVxq1" : {"label":r"$d_{0}^{ \mathrm{PV} } * \mathrm{charge}(\mu_{1}^{REC})$", "units":"cm"},
    "d0PVxq2" : {"label":r"$d_{0}^{ \mathrm{PV} } * \mathrm{charge}(\mu_{2}^{REC})$", "units":"cm"},
    
    "massZ"    : {"label":r"$m_{2\mu}$",        "units":"GeV", "default_bin_limits":[70,110,0.5], "default_x_limits":[70, 110]},
    "massZErr" : {"label":r"$\delta m_{2\mu}$", "units":"GeV", "default_bin_limits":[0,2,0.05],   "default_x_limits":[-0.1, 2.1]},

    
# Unused branches: 
# ', 'massZErr', 'massZ_vtx', 'massZ_vtx_FSR', 'massErrZ_vtx',
#        'massErrZ_vtx_FSR', 'massZ_vtxChi2', 'massZ_vtx_BS',
#        'm1', 'm2', 'Id1', 'Id2', 'Tight1', 'Tight2', 'pterr1', 'pterr2', 'weight','GENmass2l', 
#         'nFSRPhotons',  'genLep_p1', 'genLep_p2', 'p1', 'p2', 
#         'delta_Rtheta1', 'delta_Rtheta2', 'delta_pToverGenpT2'
}

In [46]:
kbin.binned_df.columns

Index(['massZ', 'massZErr', 'massZ_vtx', 'massZ_vtx_FSR', 'massErrZ_vtx',
       'massErrZ_vtx_FSR', 'massZ_vtxChi2', 'massZ_vtx_BS', 'massZ_vtx_BS_FSR',
       'massErrZ_vtx_BS', 'massErrZ_vtx_BS_FSR', 'massZ_vtxChi2_BS', 'pT1',
       'pT2', 'eta1', 'eta2', 'phi1', 'phi2', 'm1', 'm2', 'vtx_pT1', 'vtx_pT2',
       'vtx_eta1', 'vtx_eta2', 'vtx_phi1', 'vtx_phi2', 'vtx_BS_pT1',
       'vtx_BS_pT2', 'vtx_BS_eta1', 'vtx_BS_eta2', 'vtx_BS_phi1',
       'vtx_BS_phi2', 'vtx_pT_FSR1', 'vtx_pT_FSR2', 'vtx_eta_FSR1',
       'vtx_eta_FSR2', 'vtx_phi_FSR1', 'vtx_phi_FSR2', 'vtx_BS_pT_FSR1',
       'vtx_BS_pT_FSR2', 'vtx_BS_eta_FSR1', 'vtx_BS_eta_FSR2',
       'vtx_BS_phi_FSR1', 'vtx_BS_phi_FSR2', 'd0BS1', 'd0BS2', 'd0PV1',
       'd0PV2', 'Id1', 'Id2', 'Tight1', 'Tight2', 'pterr1', 'pterr2', 'weight',
       'GENmass2l', 'genLep_pt1', 'genLep_pt2', 'genLep_eta1', 'genLep_eta2',
       'genLep_phi1', 'genLep_phi2', 'nFSRPhotons', 'genLep_theta1',
       'genLep_theta2', 'theta1', 'theta2', 'genLep_

In [51]:
# def make_all_kinem_plots(kbin, iterations):
#     with PdfPages("/Users/Jake/Desktop/20200415/kinematics_and_fits/practice1.pdf") as pdf:

with PdfPages("/Users/Jake/Desktop/20200415/kinematics_and_fits/practice4.pdf") as pdf:

    fig, ax_tup, ax_ratio_tup = make_2by2_subplots_for_ratioplots()
    kbin.plot_kinem_genrec_comparison("genLep_pt1","pT1", x_range_ls=[0,150], bin_limits=[0, 120, 1], ax=ax_tup[0][0], ax_ratio=ax_ratio_tup[0][0], y_max=-1, log_scale=False)
    kbin.plot_kinem_genrec_comparison("genLep_pt2","pT2", x_range_ls=[0,150], bin_limits=[0, 120, 1], ax=ax_tup[0][1], ax_ratio=ax_ratio_tup[0][1], y_max=-1, log_scale=False)
    kbin.plot_kinem_genrec_comparison("genLep_eta1","eta1", x_range_ls=[-2.5,2.5], bin_limits=[-2.4, 2.4, 0.05], ax=ax_tup[1][0], ax_ratio=ax_ratio_tup[1][0], y_max=-1, log_scale=False)    
    kbin.plot_kinem_genrec_comparison("genLep_eta2","eta2", x_range_ls=[-2.5,2.5], bin_limits=[-2.4, 2.4, 0.05], ax=ax_tup[1][1], ax_ratio=ax_ratio_tup[1][1], y_max=-1, log_scale=False)    
    pdf.savefig()
    plt.close("all")
    
    fig, ax_tup, ax_ratio_tup = make_2by2_subplots_for_ratioplots()
    kbin.plot_kinem_genrec_comparison("genLep_eta1","eta1", x_range_ls=[-2.5,2.5], bin_limits=[-2.4, 2.4, 0.05], ax=ax_tup[0][0], ax_ratio=ax_ratio_tup[0][0], y_max=-1, log_scale=False)    
    kbin.plot_kinem_genrec_comparison("genLep_eta2","eta2", x_range_ls=[-2.5,2.5], bin_limits=[-2.4, 2.4, 0.05], ax=ax_tup[0][1], ax_ratio=ax_ratio_tup[0][1], y_max=-1, log_scale=False) 
    kbin.plot_kinem_genrec_comparison("genLep_phi1","phi1", x_range_ls=[-np.pi-0.2, np.pi+0.2], bin_limits=[-np.pi, np.pi, 0.05], ax=ax_tup[1][0], ax_ratio=ax_ratio_tup[1][0], y_max=-1, log_scale=False)
    kbin.plot_kinem_genrec_comparison("genLep_phi2","phi2", x_range_ls=[-np.pi-0.2, np.pi+0.2], bin_limits=[-np.pi, np.pi, 0.05], ax=ax_tup[1][1], ax_ratio=ax_ratio_tup[1][1], y_max=-1, log_scale=False)
    pdf.savefig()
    plt.close("all")
    
#     fig, ax_tup, ax_ratio_tup = make_2by2_subplots_for_ratioplots()
#     kbin.plot_kinem_genrec_comparison("genLep_pt1","pT1", x_range_ls=[0,150], bin_limits=[0, 120, 1], ax=ax_tup[0][0], ax_ratio=ax_ratio_tup[0][0], y_max=-1, log_scale=False)
#     kbin.plot_kinem_genrec_comparison("genLep_pt2","pT2", x_range_ls=[0,150], bin_limits=[0, 120, 1], ax=ax_tup[0][1], ax_ratio=ax_ratio_tup[0][1], y_max=-1, log_scale=False)
#     kbin.plot_kinem_genrec_comparison("genLep_eta1","eta1", x_range_ls=[-2.5,2.5], bin_limits=[-2.4, 2.4, 0.05], ax=ax_tup[1][0], ax_ratio=ax_ratio_tup[1][0], y_max=-1, log_scale=False)    
#     kbin.plot_kinem_genrec_comparison("genLep_eta2","eta2", x_range_ls=[-2.5,2.5], bin_limits=[-2.4, 2.4, 0.05], ax=ax_tup[1][1], ax_ratio=ax_ratio_tup[1][1], y_max=-1, log_scale=False)    
#     pdf.savefig()
#     plt.close()
    
#     fig = plt.figure(figsize=(28,16))
#     ax = plt.subplot(221)
#     kbin.plot_1D_kinematics(lep=1, kinem="delta_eta1", x_limits=[-0.015, 0.015], bin_limits=[-0.012, 0.012, 0.0001], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))
#     ax = plt.subplot(222)
#     kbin.plot_1D_kinematics(lep=2, kinem="delta_eta2", x_limits=[-0.015, 0.015], bin_limits=[-0.012, 0.012, 0.0001], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))
#     ax = plt.subplot(223)
#     kbin.plot_1D_kinematics(lep=1, kinem="delta_theta1", x_limits=[-0.002, 0.002], bin_limits=[-0.0018, 0.0018, 0.00002], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))
#     ax = plt.subplot(224)
#     kbin.plot_1D_kinematics(lep=2, kinem="delta_theta2", x_limits=[-0.002, 0.002], bin_limits=[-0.0018, 0.0018, 0.00002], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))
#     pdf.savefig()
#     plt.close("all")
    
    fig = plt.figure(figsize=(28,16))
    ax = plt.subplot(221)
    kbin.plot_1D_kinematics(lep=1, kinem="delta_phi1", x_limits=[-0.015, 0.015], bin_limits=[-0.012, 0.012, 0.0001], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))    
    ax = plt.subplot(222)
    kbin.plot_1D_kinematics(lep=2, kinem="delta_phi2", x_limits=[-0.015, 0.015], bin_limits=[-0.012, 0.012, 0.0001], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))
    ax = plt.subplot(223)
    kbin.plot_1D_kinematics(lep=1, kinem="delta_R1", x_limits=[-0.001, 0.012], bin_limits=[0, 0.01, 0.00005], ax=ax, y_max=-1, log_scale=False, iter_gaus=(False, 5))
    ax = plt.subplot(224)
    kbin.plot_1D_kinematics(lep=2, kinem="delta_R2", x_limits=[-0.001, 0.012], bin_limits=[0, 0.01, 0.00005], ax=ax, y_max=-1, log_scale=False, iter_gaus=(False, 5))
    pdf.savefig()
    plt.close("all")
    
    #--- Momentum plots ---#
    fig = plt.figure(figsize=(28,16))
    ax = plt.subplot(221)
    kbin.plot_1D_kinematics(lep=1, kinem="delta_pT1", x_limits=[-50, 50], bin_limits=[-30, 30, 0.5], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))
    ax = plt.subplot(222)
    kbin.plot_1D_kinematics(lep=2, kinem="delta_pT2", x_limits=[-50, 50], bin_limits=[-30, 30, 0.5], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))
#     ax = plt.subplot(223)
#     kbin.plot_1D_kinematics(lep=1, kinem="massZ", x_limits=[0,0], bin_limits=[0,0,0], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))
#     ax = plt.subplot(224)
#     kbin.plot_1D_kinematics(lep=1, kinem="massZErr", x_limits=[0,0], bin_limits=[0,0,0], ax=ax, y_max=-1, log_scale=False, iter_gaus=(False, 4))
    pdf.savefig()
    plt.close("all")
    
#     fig = plt.figure(figsize=(28,16))
#     ax = plt.subplot(221)
#     kbin.plot_1D_kinematics(lep=1, kinem="delta_pToverpT1", x_limits=[-0.5, 0.5], bin_limits=[-0.5, 0.5, 0.004], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))
#     ax = plt.subplot(222)
#     kbin.plot_1D_kinematics(lep=2, kinem="delta_pToverpT2", x_limits=[-0.5, 0.5], bin_limits=[-0.5, 0.5, 0.004], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))
#     ax = plt.subplot(223)
#     kbin.plot_1D_kinematics(lep=1, kinem="delta_pToverRecpT1", x_limits=[-0.5, 0.5], bin_limits=[-0.5, 0.5, 0.004], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))
#     ax = plt.subplot(224)
#     kbin.plot_1D_kinematics(lep=2, kinem="delta_pToverRecpT2", x_limits=[-0.5, 0.5], bin_limits=[-0.5, 0.5, 0.004], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))
#     pdf.savefig()
#     plt.close("all")

#     #--- d0 Plots ---#
#     fig = plt.figure(figsize=(28,16))
#     ax = plt.subplot(221)
#     kbin.plot_1D_kinematics(lep=1, kinem="d0BSxq1", x_limits=[-0.05, 0.05], bin_limits=[-0.02, 0.02, 0.0004], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))
#     ax = plt.subplot(222)
#     kbin.plot_1D_kinematics(lep=1, kinem="d0PVxq1", x_limits=[-0.05, 0.05], bin_limits=[-0.02, 0.02, 0.0004], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))
#     ax = plt.subplot(223)
#     kbin.plot_1D_kinematics(lep=2, kinem="d0BSxq2", x_limits=[-0.05, 0.05], bin_limits=[-0.02, 0.02, 0.0004], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))
#     ax = plt.subplot(224)
#     kbin.plot_1D_kinematics(lep=2, kinem="d0PVxq2", x_limits=[-0.05, 0.05], bin_limits=[-0.02, 0.02, 0.0004], ax=ax, y_max=-1, log_scale=False, iter_gaus=(True, 4))
#     pdf.savefig()
#     plt.close("all")
    
#     kbin.make2Dplot_dPhi_vs_dEtaANDdTheta(run_over_only_n_evts=-1,
#                                             x1_bounds=[-0.003, 0.003, 0.00003], 
#                                             x2_bounds=[-0.003, 0.003, 0.00003], 
#                                             y_bounds=[-0.003, 0.003, 0.00003], 
#                                             exclusive=True,
#                                             save_plot=False, 
#                                             save_as_png=False,
#                                             verbose=False,
#                                             outpath="/Users/Jake/Desktop/20200412/2Dplots_dphi_vs_deta_TESTING/"
#                                             )
#     saveit(pdf)
    
    plt.close("all")

Performing 4 iterative Gaussian fits
Performing 4 iterative Gaussian fits
Performing 4 iterative Gaussian fits
Performing 4 iterative Gaussian fits


In [37]:
np.sum(kbin.binned_df['massZ'] == 0)

0

In [ ]:
def saveit(pdf):
    pdf.savefig()
    plt.close()
    

In [ ]:
with PdfPages("/Users/Jake/Desktop/THIS_IS_SO_AMAZING.pdf") as pdf:
    #--- Angular plots ---#
    fig = plt.figure(figsize=(28,16))
    ax = plt.subplot(221)
    kbin.plot_1D_kinematics(lep=1, kinem="delta_eta1", x_limits=[-0.015, 0.015], bin_limits=[-0.012, 0.012, 0.0001], ax=ax, y_max=-1, log_scale=False)
    ax = plt.subplot(222)
    kbin.plot_1D_kinematics(lep=1, kinem="delta_theta1", x_limits=[-0.015, 0.015], bin_limits=[-0.012, 0.012, 0.0001], ax=ax, y_max=-1, log_scale=False)
    ax = plt.subplot(223)
    kbin.plot_1D_kinematics(lep=1, kinem="delta_phi1", x_limits=[-0.015, 0.015], bin_limits=[-0.012, 0.012, 0.0001], ax=ax, y_max=-1, log_scale=False)
    ax = plt.subplot(224)
    kbin.plot_1D_kinematics(lep=1, kinem="delta_R1", x_limits=[-0.001, 0.012], bin_limits=[0, 0.01, 0.00005], ax=ax, y_max=-1, log_scale=False)
    pdf.savefig()
    plt.close()
    
    fig = plt.figure(figsize=(28,16))
    ax = plt.subplot(221)
    kbin.plot_1D_kinematics(lep=2, kinem="delta_eta2", x_limits=[-0.015, 0.015], bin_limits=[-0.012, 0.012, 0.0001], ax=ax, y_max=-1, log_scale=False)
    ax = plt.subplot(222)
    kbin.plot_1D_kinematics(lep=2, kinem="delta_theta2", x_limits=[-0.015, 0.015], bin_limits=[-0.012, 0.012, 0.0001], ax=ax, y_max=-1, log_scale=False)
    ax = plt.subplot(223)
    kbin.plot_1D_kinematics(lep=2, kinem="delta_phi2", x_limits=[-0.015, 0.015], bin_limits=[-0.012, 0.012, 0.0001], ax=ax, y_max=-1, log_scale=False)
    ax = plt.subplot(224)
    kbin.plot_1D_kinematics(lep=2, kinem="delta_R2", x_limits=[-0.001, 0.012], bin_limits=[0, 0.01, 0.00005], ax=ax, y_max=-1, log_scale=False)
    pdf.savefig()
    plt.close()
    
    #--- Momentum plots ---#
    fig = plt.figure(figsize=(28,16))
    ax = plt.subplot(221)
    kbin.plot_1D_kinematics(lep=1, kinem="delta_pT1", x_limits=[-50, 50], bin_limits=[-30, 30, 0.1], ax=ax, y_max=-1, log_scale=False)
    ax = plt.subplot(223)
    kbin.plot_1D_kinematics(lep=1, kinem="delta_pToverpT1", x_limits=[-0.5, 0.5], bin_limits=[-0.5, 0.5, 0.002], ax=ax, y_max=-1, log_scale=False)
    ax = plt.subplot(224)
    kbin.plot_1D_kinematics(lep=1, kinem="delta_pToverRecpT1", x_limits=[-0.5, 0.5], bin_limits=[-0.5, 0.5, 0.002], ax=ax, y_max=-1, log_scale=False)
    pdf.savefig()
    plt.close()
    
    fig = plt.figure(figsize=(28,16))
    ax = plt.subplot(221)
    kbin.plot_1D_kinematics(lep=2, kinem="delta_pT2", x_limits=[-50, 50], bin_limits=[-30, 30, 0.1], ax=ax, y_max=-1, log_scale=False)
    ax = plt.subplot(223)
    kbin.plot_1D_kinematics(lep=2, kinem="delta_pToverpT2", x_limits=[-0.5, 0.5], bin_limits=[-0.5, 0.5, 0.002], ax=ax, y_max=-1, log_scale=False)
    ax = plt.subplot(224)
    kbin.plot_1D_kinematics(lep=2, kinem="delta_pToverRecpT2", x_limits=[-0.5, 0.5], bin_limits=[-0.5, 0.5, 0.002], ax=ax, y_max=-1, log_scale=False)
    pdf.savefig()
    plt.close()

    #--- d0 Plots ---#
    fig = plt.figure(figsize=(28,16))
    ax = plt.subplot(221)
    kbin.plot_1D_kinematics(lep=1, kinem="d0BS1", x_limits=[-0.05, 0.05], bin_limits=[-0.02, 0.02, 0.0004], ax=ax, y_max=-1, log_scale=False)
    ax = plt.subplot(222)
    kbin.plot_1D_kinematics(lep=1, kinem="d0PV1", x_limits=[-0.05, 0.05], bin_limits=[-0.02, 0.02, 0.0004], ax=ax, y_max=-1, log_scale=False)
    ax = plt.subplot(223)
    kbin.plot_1D_kinematics(lep=2, kinem="d0BS2", x_limits=[-0.05, 0.05], bin_limits=[-0.02, 0.02, 0.0004], ax=ax, y_max=-1, log_scale=False)
    ax = plt.subplot(224)
    kbin.plot_1D_kinematics(lep=2, kinem="d0PV2", x_limits=[-0.05, 0.05], bin_limits=[-0.02, 0.02, 0.0004], ax=ax, y_max=-1, log_scale=False)
    pdf.savefig()
    plt.close("all")

# Put Reco-Gen Comparison plots in one PDF:

In [ ]:
with PdfPages("/Users/Jake/Desktop/kinem_genrec_plots4.pdf") as pdf:
    fig, ax_tup, ax_ratio_tup = make_2by2_subplots_for_ratioplots()
    kbin.plot_kinem_genrec_comparison("genLep_pt1","pT1", x_range_ls=[0,150], bin_limits=[0, 120, 1], ax=ax_tup[0][0], ax_ratio=ax_ratio_tup[0][0], y_max=-1, log_scale=False)
    kbin.plot_kinem_genrec_comparison("genLep_pt2","pT2", x_range_ls=[0,150], bin_limits=[0, 120, 1], ax=ax_tup[0][1], ax_ratio=ax_ratio_tup[0][1], y_max=-1, log_scale=False)
    kbin.plot_kinem_genrec_comparison("genLep_eta1","eta1", x_range_ls=[-2.5,2.5], bin_limits=[-2.4, 2.4, 0.05], ax=ax_tup[1][0], ax_ratio=ax_ratio_tup[1][0], y_max=-1, log_scale=False)    
    kbin.plot_kinem_genrec_comparison("genLep_eta2","eta2", x_range_ls=[-2.5,2.5], bin_limits=[-2.4, 2.4, 0.05], ax=ax_tup[1][1], ax_ratio=ax_ratio_tup[1][1], y_max=-1, log_scale=False)    
    pdf.savefig()
    plt.close()

# $\Delta \phi$ vs. $\Delta \eta$ AND $\Delta \theta$ 2D plots.

#### Loop over $\Delta \phi$ vs. $(\Delta \eta, \Delta \theta)$ plots --- or ---make a PDF

In [ ]:
%%time
# %%capture

n_evts_scan = 10000000
n_evts_keep = 5000

# eta_bin_edges = [0.00, 0.30] # barrel
# eta_bin_edges = [0.80, 1.10]  # overlap
eta_bin_edges = [2.10, 2.40]  # endcap

# eta_bin_edges = [0.00, 0.10, 0.20, 0.30]    # barrel
# eta_bin_edges = [0.70, 0.80, 0.90, 1.00, 1.10, 1.20]    # overlap
# eta_bin_edges = [2.00, 2.10, 2.20, 2.30, 2.40]    # endcap

#--- Could be either pT or p. User specifies when initializing kbin. ---#
# p_bin_edges = [5, 20, 30, 40, 50, 60, 100]
# p_bin_edges = [5, 7, 10, 15, 20, 25, 30, 35, 40, 45, 50, 100]
# p_bin_edges = [5, 7, 10, 15, 20]
p_bin_edges = [20, 40, 60, 80, 100]
use_ptotal_instead = False  # p_total or pT

outpath = "/Users/Jake/Desktop/20200410/2Dplots_dPhivsdEtaANDdTheta/"
pdf_name_base = "combinedpdf_endcap_highpT"

save_plot = False  # Save individual plots.
save_as_png = False
save_pdf = True  
verbose = True

if (save_plot) and (save_pdf or save_as_png):
    err_msg = "PROGRAM STOPPED: both 'save_plot' and 'save_pdf' are True. Make one of them False."
    raise RuntimeError(err_msg)

makeDirs(outpath)
all_kbin_ls = []
for k in range(len(eta_bin_edges)-1):
    this_eta = eta_bin_edges[k]
    next_eta = eta_bin_edges[k+1]
    
    kbin_ls = []
    for m in range(len(p_bin_edges)-1):
        this_p = p_bin_edges[m]
        next_p = p_bin_edges[m+1]

        kbin = KinemBinnedEtaPt(df_MC_2016, 
                                n_evts=n_evts_scan, 
                                eta_cut_ls=[this_eta, next_eta], 
                                pT_cut_ls=[this_p, next_p], 
                                use_ptotal_instead=False, 
                                dR_cut=0.02, verbose=verbose)
        kbin_ls.append(kbin)
        
    # Finished looping over p bins.
    evts_max = n_evts_keep

    if save_pdf and not save_plot:
        # Save plots into one PDF:
        pT_min = min( [kb.pT_min for kb in kbin_ls] )
        pT_max = max( [kb.pT_max for kb in kbin_ls] )
        n_plots = len(kbin_ls)

        title_str_pT_min = f"{pT_min}" if pT_min < 10 else f"{pT_min}"  # For plot-ordering purposes.
        pdf_name = pdf_name_base + f"__{this_eta}_eta_{next_eta}"# + f"__{title_str_pT_min}_pT_{pT_max}"
        pdf_name += f"__{pT_min}_pT_{pT_max}"# + f"__{title_str_pT_min}_pT_{pT_max}"
        pdf_name += f"__{n_plots}plots"
        pdf_name = make_str_title_friendly(pdf_name) + ".pdf"

        outfile = os.path.join(outpath, pdf_name)

        with PdfPages(outfile) as pdf:
            for kbinned in kbin_ls:
                kbinned.make2Dplot_dPhi_vs_dEtaANDdTheta(run_over_only_n_evts=evts_max,
                                                           x1_bounds=[-0.003, 0.003, 0.00003], 
                                                           x2_bounds=[-0.003, 0.003, 0.00003], 
                                                           y_bounds=[-0.003, 0.003, 0.00003], 
                                                           exclusive=True,
                                                           save_plot=save_plot, 
                                                           save_as_png=save_as_png,
                                                           verbose=verbose,
                                                           outpath=outpath
                                                           )

                pdf.savefig()  # saves the current figure into a pdf page
                plt.close()

        print("[INFO] PDF created at", outfile, "\n")

        all_kbin_ls.append(kbin_ls)

        plt.close('all')
        
    elif save_plot and not save_pdf:
        # Save individual plots:
        for kbinned in kbin_ls:
            kbinned.make2Dplot_dPhi_vs_dEtaANDdTheta(run_over_only_n_evts=evts_max,
                                                       x1_bounds=[-0.003, 0.003, 0.00003], 
                                                       x2_bounds=[-0.003, 0.003, 0.00003], 
                                                       y_bounds=[-0.003, 0.003, 0.00003], 
                                                       exclusive=True,
                                                       save_plot=save_plot, 
                                                       save_as_png=save_as_png,
                                                       verbose=verbose,
                                                       outpath=outpath
                                                       )
            plt.close('all')

### $\Delta \phi$ vs. $\Delta \eta$ 2D plots.

In [ ]:
%%time
%config InlineBackend.figure_format ='retina'

kbin_0eta2p4_40pT60 = KinemBinnedEtaPt(df_MC_2016, n_evts=10000, eta_cut_ls=[0, 0.8], pT_cut_ls=[40, 60], dR_cut=0.002)
# kbin_0eta2p4_40pT60.make2Dplot_pT_vs_eta(eta_2D_limits=[-2.5, 2.5, 0.1], pT_2D_limits=[0, 100, 1], save_plot=False, outpath="/Users/Jake/Desktop/20200403/")
kbin_0eta2p4_40pT60.make2Dplot_dphivsdeta(x_2D_limits=[-0.005, 0.005, 0.00005], y_2D_limits=[-0.0025, 0.0025, 0.00005], save_plot=False, outpath="/Users/Jake/Desktop/20200403/2Dplots_dphi_vs_deta/")

### Loop over $p_{T}$ vs. $\eta$ plots

In [ ]:
n_evts = 100000
eta_bin_edges = [0.0, 0.9, 1.4, 2.4] 
# eta_bin_edges = [0.9, 1.4, 2.4] 
# eta_bin_edges = [0.9, 1.4, 2.4] 
pT_bin_edges = [5, 20, 30, 40, 50, 60, 100]
# pT_bin_edges = [30, 40, 50]
# pT_bin_edges = [40, 50]
eta_2D_limits = [-2.5, 2.5, 0.1] 
pT_2D_limits = [0, 100, 1] 
save_plot = True 
outpath = "/Users/Jake/Desktop/20200403/2Dplots_pT_vs_eta_1E5evts/"
                            
for k in range(len(eta_bin_edges)-1):
    this_eta = eta_bin_edges[k]
    next_eta = eta_bin_edges[k+1]

    for m in range(len(pT_bin_edges)-1):
        this_pT = pT_bin_edges[m]
        next_pT = pT_bin_edges[m+1]

        df_kinembinned = KinemBinnedDataFrame(df_MC_2016, n_evts=n_evts, eta_cut_ls=[this_eta, next_eta], pT_cut_ls=[this_pT, next_pT], dR_cut=0.002)
        df_kinembinned.make2Dplot_pT_vs_eta(eta_2D_limits=eta_2D_limits, pT_2D_limits=pT_2D_limits, save_plot=True, outpath=outpath)

# Make a PDF of 1 kinematic bin: all the distributions!

In [ ]:
%%time
# %%capture

n_evts_scan = 1000000
n_evts_keep = 5000

eta_bin_edges = [0.00, 0.20] # barrel
# eta_bin_edges = [0.80, 1.10]  # overlap
# eta_bin_edges = [2.10, 2.40]  # endcap

# eta_bin_edges = [0.00, 0.10, 0.20, 0.30]    # barrel
# eta_bin_edges = [0.70, 0.80, 0.90, 1.00, 1.10, 1.20]    # overlap
# eta_bin_edges = [2.00, 2.10, 2.20, 2.30, 2.40]    # endcap

#--- Could be either pT or p. User specifies when initializing kbin. ---#
# p_bin_edges = [5, 20, 30, 40, 50, 60, 100]
# p_bin_edges = [5, 7, 10, 15, 20, 25, 30, 35, 40, 45, 50, 100]
# p_bin_edges = [5, 7, 10, 15, 20]
p_bin_edges = [20, 40, 60, 80, 100]
use_ptotal_instead = False  # p_total or pT

outpath = "/Users/Jake/Desktop/20200410/2Dplots_dPhivsdEtaANDdTheta/"
pdf_name_base = "combinedpdf_endcap_highpT"

save_plot = False  # Save individual plots.
save_as_png = False
save_pdf = True  
verbose = True

if (save_plot) and (save_pdf or save_as_png):
    err_msg = "PROGRAM STOPPED: both 'save_plot' and 'save_pdf' are True. Make one of them False."
    raise RuntimeError(err_msg)

makeDirs(outpath)
all_kbin_ls = []
for k in range(len(eta_bin_edges)-1):
    this_eta = eta_bin_edges[k]
    next_eta = eta_bin_edges[k+1]
    
    kbin_ls = []
    for m in range(len(p_bin_edges)-1):
        this_p = p_bin_edges[m]
        next_p = p_bin_edges[m+1]

        kbin = KinemBinnedEtaPt(df_MC_2016, 
                                n_evts=n_evts_scan, 
                                eta_cut_ls=[this_eta, next_eta], 
                                pT_cut_ls=[this_p, next_p], 
                                use_ptotal_instead=False, 
                                dR_cut=0.02, verbose=verbose)
        kbin_ls.append(kbin)
        
    # Finished looping over p bins.
    evts_max = n_evts_keep

    if save_pdf and not save_plot:
        # Save plots into one PDF:
        pT_min = min( [kb.pT_min for kb in kbin_ls] )
        pT_max = max( [kb.pT_max for kb in kbin_ls] )
        n_plots = len(kbin_ls)

        title_str_pT_min = f"{pT_min}" if pT_min < 10 else f"{pT_min}"  # For plot-ordering purposes.
        pdf_name = pdf_name_base + f"__{this_eta}_eta_{next_eta}"# + f"__{title_str_pT_min}_pT_{pT_max}"
        pdf_name += f"__{pT_min}_pT_{pT_max}"# + f"__{title_str_pT_min}_pT_{pT_max}"
        pdf_name += f"__{n_plots}plots"
        pdf_name = make_str_title_friendly(pdf_name) + ".pdf"

        outfile = os.path.join(outpath, pdf_name)

        with PdfPages(outfile) as pdf:
            for kbinned in kbin_ls:
                kbinned.make2Dplot_dPhi_vs_dEtaANDdTheta(run_over_only_n_evts=evts_max,
                                                           x1_bounds=[-0.003, 0.003, 0.00003], 
                                                           x2_bounds=[-0.003, 0.003, 0.00003], 
                                                           y_bounds=[-0.003, 0.003, 0.00003], 
                                                           exclusive=True,
                                                           save_plot=save_plot, 
                                                           save_as_png=save_as_png,
                                                           verbose=verbose,
                                                           outpath=outpath
                                                           )

                kbinned.make_1D_dist_dPhi()
                kbinned.make_1D_dist_dEta()
                kbinned.make_1D_dist_dR()
                kbinned.make_1D_dist_dTheta()

                pdf.savefig()  # saves the current figure into a pdf page
                plt.close()

        print("[INFO] PDF created at", outfile, "\n")

        all_kbin_ls.append(kbin_ls)

        plt.close('all')
        
    elif save_plot and not save_pdf:
        # Save individual plots:
        for kbinned in kbin_ls:
            kbinned.make2Dplot_dPhi_vs_dEtaANDdTheta(run_over_only_n_evts=evts_max,
                                                       x1_bounds=[-0.003, 0.003, 0.00003], 
                                                       x2_bounds=[-0.003, 0.003, 0.00003], 
                                                       y_bounds=[-0.003, 0.003, 0.00003], 
                                                       exclusive=True,
                                                       save_plot=save_plot, 
                                                       save_as_png=save_as_png,
                                                       verbose=verbose,
                                                       outpath=outpath
                                                       )
            plt.close('all')